In [36]:
# BLOCK 1: Load All Files and Print Columns Neatly

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from tabulate import tabulate
import os

# Set paths
merged_log_path = r'E:\FYP\FYP Symposium\Merged Log (49).xlsx'
stock_data_path = r'E:\FYP\FYP Symposium\Trading Simulation Experiment Data Turn Wise.xlsx'
dta_path = r'E:\FYP\FYP Symposium\SurveysClean.dta'

# Load Data
merged_log = pd.read_excel(merged_log_path)
stock_xls = pd.ExcelFile(stock_data_path)
stock_tables = {stock: pd.read_excel(stock_xls, sheet_name=stock) for stock in ['TSLA', 'XOM', 'NFLX', 'PG']}
strategy = pd.read_stata(dta_path)

# Rename columns in each stock sheet
rename_map = {
    'trend': 'Close_price_diff',
    'trend direction': 'price_trend_1',
    'trend summary': 'price_trend_7',
    'volume trend change': 'volume_diff',
    'volume trend direction': 'volume_trend_1',
    'volume trend summary': 'volume_trend_7',
    'Technical Decision': 'MACD_trend',
    'Bollinger Classification': 'bollinger_trend'
}

for stock_name, df in stock_tables.items():
    stock_tables[stock_name].rename(columns=rename_map, inplace=True)


# # Show loaded columns for verification (pretty print using tabulate)
# print("Merged Log Columns:\n")
# print(tabulate([[col] for col in merged_log.columns], headers=["Merged Log Columns"], tablefmt="github"))

# print("\nStrategy Columns:\n")
# print(tabulate([[col] for col in strategy.columns], headers=["Strategy Columns"], tablefmt="github"))

print(merged_log.head(15))

          Date Real Time Simulation Time  Seconds left Participant_ID  turn action ticker News Sentiment News Truth  quantity  Stock Price  Total Stock  cash_before  cash_after  stockportfolio_before  stockportfolio_after  Total_assets  total_TSLA_holding  total_XOM_holding  total_NFLX_holding  total_PG_holding  TSLA_value  XOM_value  NFLX_value  PG_value
0   2025-04-23  10:50:33         0:00:55             5          E0070     1    Buy   TSLA       Negative      False        10       337.80      3378.00     10000.00     6622.00                   0.00               3378.00      10000.00                  10                  0                   0                 0      3378.0        0.0        0.00      0.00
1   2025-04-23  10:51:32         0:01:54            10          E0070     2   Sell   TSLA        No News    No News        10       372.00      3720.00      6622.00    10342.00                3720.00                  0.00      10342.00                   0                  0          

In [ ]:
# BLOCK 2: Rename Columns for Consistency
import pandas as pd
from tabulate import tabulate
# Rename Merged Log Columns to match our expectations
merged_log.rename(columns={
    'News Sentiment': 'news_sentiment',
    'News Truth': 'news_truth'
}, inplace=True)

# Rename Strategy Columns if needed
if 'participant_id' not in strategy.columns:
    if 'ResponseID' in strategy.columns:
        strategy.rename(columns={'ResponseID': 'participant_id'}, inplace=True)
    elif 'ResponseId' in strategy.columns:
        strategy.rename(columns={'ResponseId': 'participant_id'}, inplace=True)

# Define scoring function
def assign_score(row, method_name):
    score = 0
    for i, weight in zip(['1st', '2nd', '3rd'], [3, 2, 1]):
        factor = row.get(f'DecisionFactor_{i}', None)
        if factor == method_name:
            score = weight
            break
    return score

def map_decision_bias(score):
    if score == 2.5:
        return -1
    elif score == 2:
        return 0
    elif score == 1.5:
        return 1
    else:
        return None  # or np.nan if preferred


# Add scoring for 'Graph'
strategy['Graph Scoring'] = strategy.apply(lambda row: assign_score(row, 'Graph'), axis=1)

# Add scoring for 'Data Table'
strategy['Data Table Scoring'] = strategy.apply(lambda row: assign_score(row, 'Data Table'), axis=1)

# Add column for average of Graph Scoring and Data Table Scoring
strategy['Average Scoring'] = strategy[['Graph Scoring', 'Data Table Scoring']].mean(axis=1)

strategy['Data Penalty'] = strategy['Average Scoring'].apply(map_decision_bias)

# View result
print(strategy[['Participant_ID', 'DecisionFactor_1st', 'DecisionFactor_2nd', 'DecisionFactor_3rd',
                'Graph Scoring', 'Data Table Scoring', 'Average Scoring', 'Data Penalty']].head(10))

# Columns to export
columns_to_export = [
    'Participant_ID', 'DecisionFactor_1st', 'DecisionFactor_2nd', 'DecisionFactor_3rd',
    'Graph Scoring', 'Data Table Scoring', 'Average Scoring', 'Data Penalty'
]

# Clean copy with selected columns
export_strategy = strategy[columns_to_export].copy()



  Participant_ID DecisionFactor_1st DecisionFactor_2nd DecisionFactor_3rd  Graph Scoring  Data Table Scoring  Average Scoring  Data Penalty
0          E0070              Graph         Data Table      News Headline              3                   2              2.5            -1
1          E0169              Graph      News Headline         Data Table              3                   1              2.0             0
2          E0426      News Headline         Data Table              Graph              1                   2              1.5             1
3          E0712      News Headline              Graph         Data Table              2                   1              1.5             1
4          E1130              Graph      News Headline         Data Table              3                   1              2.0             0
5          E1217      News Headline              Graph         Data Table              2                   1              1.5             1
6          E1719    

In [ ]:
# BLOCK 3: Knowledge Scoring

def check_knowledge(prm2b_14a_response):
    if pd.isna(prm2b_14a_response):
        return {'MACD': False, 'Bollinger': False, 'Volume': False, 'HighLow_OpenClose': False}

    text = prm2b_14a_response.lower()
    
    knowledge = {
        'MACD': False,
        'Bollinger': False,
        'Volume': False,
        'HighLow_OpenClose': False
    }

    # Check MACD knowledge
    if 'macd' in text:
        knowledge['MACD'] = True
    
    # Check Bollinger Bands
    if 'bollinger' in text or 'bb' in text:
        knowledge['Bollinger'] = True

    # Check Volume
    if 'volume' in text:
        knowledge['Volume'] = True

    # Check High Low Open Close
    if 'high' in text or 'low' in text or 'open' in text or 'close' in text or 'ohlc' in text:
        knowledge['HighLow_OpenClose'] = True

    return knowledge

# --- Apply to all participants ---

knowledge_records = []

for idx, row in strategy.iterrows():
    participant_id = row['Participant_ID']
    prm2b_14a = row.get('PRM2b_14a', None)
    knowledge = check_knowledge(prm2b_14a)

    knowledge_records.append({
        'Participant_ID': participant_id,
        'Knows_MACD': 'Yes' if knowledge['MACD'] else 'No',
        'Knows_Bollinger': 'Yes' if knowledge['Bollinger'] else 'No',
        'Knows_Volume': 'Yes' if knowledge['Volume'] else 'No',
        'Knows_HighLow_OpenClose': 'Yes' if knowledge['HighLow_OpenClose'] else 'No'
    })

# Create DataFrame
knowledge_df = pd.DataFrame(knowledge_records)

# Drop 'Knows_HighLow_OpenClose' — no longer needed
if 'Knows_HighLow_OpenClose' in knowledge_df.columns:
    knowledge_df.drop(columns=['Knows_HighLow_OpenClose'], inplace=True)

# --- Merge Cleaned Knowledge into Strategy ---

# Drop old versions to avoid MergeError
strategy.drop(columns=['Knows_MACD', 'Knows_Bollinger', 'Knows_Volume', 'Knows_HighLow_OpenClose', 'Knowledge_Score'], errors='ignore', inplace=True)

# Merge new knowledge
strategy = pd.merge(strategy, knowledge_df, on='Participant_ID', how='left')
print("\n✅ Merged updated knowledge into strategy successfully.")

# --- Scoring Logic ---

# Map Yes/No with new scoring logic (Volume: Yes = 1, No = 0.5)
knowledge_df_numeric = knowledge_df.copy()
knowledge_df_numeric['Knows_MACD'] = knowledge_df_numeric['Knows_MACD'].map({'Yes': 1, 'No': 0})
knowledge_df_numeric['Knows_Bollinger'] = knowledge_df_numeric['Knows_Bollinger'].map({'Yes': 1, 'No': 0})
knowledge_df_numeric['Knows_Volume'] = knowledge_df_numeric['Knows_Volume'].map({'Yes': 1, 'No': 0.5})

# Compute score
knowledge_df['Knowledge_Score'] = (
    knowledge_df_numeric['Knows_MACD'] +
    knowledge_df_numeric['Knows_Bollinger'] +
    knowledge_df_numeric['Knows_Volume']
)

# Show sample output
print("\n✅ Final Knowledge Score (range 0.5–3.0):")
display(knowledge_df[['Participant_ID', 'Knows_MACD', 'Knows_Bollinger', 'Knows_Volume', 'Knowledge_Score']].head())

# Optional documentation string
knowledge_score_note = "Knowledge_Score = Knows_MACD (1/0) + Knows_Bollinger (1/0) + Knows_Volume (1 if Yes, 0.5 if No); Range = 0.5 to 3.0"

# Export to Excel
output_knowledge_path = r'E:\FYP\FYP Symposium\Outputs\Knowledge_Score_Updated.xlsx'
knowledge_df.to_excel(output_knowledge_path, index=False)
print(f"\n✅ Updated Knowledge Data (with Knowledge_Score) exported to: {output_knowledge_path}")



✅ Merged updated knowledge into strategy successfully.

✅ Final Knowledge Score (range 0.5–3.0):


,Participant_ID,Knows_MACD,Knows_Bollinger,Knows_Volume,Knowledge_Score
0,E0070,No,No,No,0.5
1,E0169,Yes,Yes,Yes,3.0
2,E0426,No,No,No,0.5
3,E0712,No,No,No,0.5
4,E1130,Yes,No,No,1.5



✅ Updated Knowledge Data (with Knowledge_Score) exported to: E:\FYP\FYP Symposium\Outputs\Knowledge_Score_Updated.xlsx


In [ ]:
#=========== block 4.7b ================

import pandas as pd
from IPython.display import display, HTML
pd.set_option('display.max_columns', None)   # Show all columns
pd.set_option('display.width', 1000)   

def process_turn_data(filepath):
    xls = pd.ExcelFile(filepath)
    processed_sheets = {}

    for sheet_name in xls.sheet_names:
        df = xls.parse(sheet_name)
        df_proc = df.copy()

        # MACD Interpretation
        def interpret_macd(row):
            macd, signal, hist = row["MACD (12,26,9)"], row["Signal (12,26,9)"], row["MACD Histogram (12,26,9)"]
            if macd > signal and hist > 0:
                return "Buy", "MACD is above Signal with positive momentum (Histogram > 0)."
            elif macd < signal and hist < 0:
                return "Sell", "MACD is below Signal with negative momentum (Histogram < 0)."
            else:
                return "Hold", "MACD shows weak or unclear trend."
        df_proc[["MACD Signal", "MACD Explanation"]] = df_proc.apply(interpret_macd, axis=1, result_type="expand")
        df_proc["MACD Calc (MACD - Signal)"] = df_proc["MACD (12,26,9)"] - df_proc["Signal (12,26,9)"]

        # Bollinger Bands Interpretation
        def interpret_bollinger(row):
            close = row["Close"]
            top, bottom = row["Top Bollinger Bands (20,O,2,ma,n)"], row["Bottom Bollinger Bands (20,O,2,ma,n)"]
            pct = (close - bottom) / (top - bottom) if (top - bottom) != 0 else 0.5
            if pct < 0.25:
                return pct, "Buy", "Close near Bottom Bollinger Band; likely rebound."
            elif pct > 0.75:
                return pct, "Sell", "Close near Top Bollinger Band; likely pullback."
            else:
                return pct, "Hold", "Close near Median; market balanced."
        df_proc[["Bollinger Position %", "Bollinger Signal", "Bollinger Explanation"]] = df_proc.apply(
            interpret_bollinger, axis=1, result_type="expand"
        )

        # Volume Analysis
        df_proc["Volume 7d Avg"] = df_proc["Volume"].rolling(window=7, min_periods=1).mean()
        df_proc["Volume Strength"] = df_proc["Volume"] / df_proc["Volume 7d Avg"]
        def volume_logic(row):
            strength = row["Volume Strength"]
            if strength > 1.25:
                return strength, "Buy", "High volume relative to 7-day avg; strong interest."
            elif strength < 0.75:
                return strength, "Sell", "Low volume; weak interest."
            else:
                return strength, "Hold", "Normal volume; no strong signal."
        df_proc[["Volume Strength", "Volume Signal", "Volume Explanation"]] = df_proc.apply(
            volume_logic, axis=1, result_type="expand"
        )

        # Candlestick Pattern Interpretation
        def interpret_candlestick(row):
            open_, close = row["Open"], row["Close"]
            high, low = row["High"], row["Low"]
            volatility = high - low
            if close > open_:
                return volatility, "Buy", "Bullish candle (Close > Open).", "Based on Open vs Close (100%)", 1
            elif close < open_:
                return volatility, "Sell", "Bearish candle (Close < Open).", "Based on Open vs Close (100%)", -1
            else:
                return volatility, "Hold", "Neutral candle.", "Based on Open vs Close (100%)", 0
        df_proc[[ 
            "Volatility (High - Low)", "Candle Signal", "Candle Explanation",
            "Candle Signal Explanation", "Candle Score"
        ]] = df_proc.apply(interpret_candlestick, axis=1, result_type="expand")
        
        # Scaled Numeric Scores
        df_proc["MACD Score"] = df_proc["MACD (12,26,9)"] - df_proc["Signal (12,26,9)"]
        df_proc["Bollinger Score"] = (df_proc["Bollinger Position %"] - 0.5) * 2
        df_proc["Volume Score"] = df_proc["Volume Strength"] - 1

        vol_min = df_proc["Volatility (High - Low)"].min()
        vol_max = df_proc["Volatility (High - Low)"].max()
        df_proc["Candle Weighted Score"] = df_proc["Candle Score"] * (
            (df_proc["Volatility (High - Low)"] - vol_min) / (vol_max - vol_min + 1e-9)
        )
        
        # # Add RSI (14) and Momentum (10) calculations
        # delta = df_proc["Close"].diff()
        # gain = delta.where(delta > 0, 0)
        # loss = -delta.where(delta < 0, 0)

        # avg_gain = gain.rolling(window=14, min_periods=14).mean()
        # avg_loss = loss.rolling(window=14, min_periods=14).mean()
        # rs = avg_gain / (avg_loss + 1e-9)
        # df_proc["RSI (14)"] = 100 - (100 / (1 + rs))

        # df_proc["Momentum (10)"] = df_proc["Close"] - df_proc["Close"].shift(10)

        # # Add EMA crossover logic (12 EMA vs 26 EMA)
        # df_proc["EMA 12"] = df_proc["Close"].ewm(span=12, adjust=False).mean()
        # df_proc["EMA 26"] = df_proc["Close"].ewm(span=26, adjust=False).mean()
        # df_proc["EMA Signal"] = df_proc.apply(
        #     lambda row: "Buy" if row["EMA 12"] > row["EMA 26"] else (
        #         "Sell" if row["EMA 12"] < row["EMA 26"] else "Hold"
        #     ),
        #     axis=1
        # )


        # Rearranging Columns
        desired_order = [
            # Basic OHLCV
            "Turn", "Date", "Open", "High", "Low", "Close", "Volume", "Volume 7d Avg", "Volume Strength", "Volume Signal", "Volume Explanation", "Volume Score",
            "Volatility (High - Low)", "Candle Signal", "Candle Explanation",
            "Candle Signal Explanation", "Candle Score", "Candle Weighted Score",
            # MACD group
            "MACD (12,26,9)", "Signal (12,26,9)", "MACD Histogram (12,26,9)",
            "MACD Signal", "MACD Calc (MACD - Signal)", "MACD Explanation", "MACD Score",
            # Bollinger group
            "Top Bollinger Bands (20,O,2,ma,n)", "Median Bollinger Bands (20,O,2,ma,n)", "Bottom Bollinger Bands (20,O,2,ma,n)",
            "Bollinger Position %", "Bollinger Signal", "Bollinger Explanation", "Bollinger Score"

        ]

        # Only include columns that actually exist (some might be missing in some sheets)
        existing_cols = [col for col in desired_order if col in df_proc.columns]
        remaining_cols = [col for col in df_proc.columns if col not in existing_cols]
        df_proc = df_proc[existing_cols + remaining_cols]

                # Final Prediction Columns
        def compute_final_prediction(row):
            w_macd = 1.0
            w_boll = 1.0
            w_vol = 0.8
            w_candle = 0.6

            score = (
                w_macd * row["MACD Score"] +
                w_boll * row["Bollinger Score"] +
                w_vol * row["Volume Score"] +
                w_candle * row["Candle Weighted Score"]
            )

            if score > 1:
                action = "Buy"
            elif score < -1:
                action = "Sell"
            else:
                action = "Hold"

            explanation = (
                f"Final Score: {score:.2f} | "
                f"MACD: {w_macd}×{row['MACD Score']:.2f}, "
                f"Bollinger: {w_boll}×{row['Bollinger Score']:.2f}, "
                f"Volume: {w_vol}×{row['Volume Score']:.2f}, "
                f"Candle: {w_candle}×{row['Candle Weighted Score']:.2f}"
            )
            return pd.Series([score, action, explanation])

        df_proc[["Final Score", "Predicted Action", "Prediction Explanation"]] = df_proc.apply(
            compute_final_prediction, axis=1
        )

        processed_sheets[sheet_name] = df_proc

    return processed_sheets

file_path = r'E:\FYP\FYP Symposium\Turn Data in descending order.xlsx'
processed = process_turn_data(file_path)
print(processed["PG"].head())

# Export to a new Excel file
output_path = r'E:\FYP\FYP Symposium\Indicator predictions\turns indicator market prediction.xlsx'
with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
    for sheet_name, df in processed.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Processed data exported to: {output_path}")

   Turn  Date    Open    High     Low   Close   Volume  Volume 7d Avg  Volume Strength Volume Signal                Volume Explanation  Volume Score  Volatility (High - Low) Candle Signal              Candle Explanation      Candle Signal Explanation  Candle Score  Candle Weighted Score  MACD (12,26,9)  Signal (12,26,9)  MACD Histogram (12,26,9) MACD Signal  MACD Calc (MACD - Signal)                                   MACD Explanation  MACD Score  Top Bollinger Bands (20,O,2,ma,n)  Median Bollinger Bands (20,O,2,ma,n)  Bottom Bollinger Bands (20,O,2,ma,n)  Bollinger Position % Bollinger Signal                            Bollinger Explanation  Bollinger Score  Final Score Predicted Action                             Prediction Explanation
0     6     4  163.44  165.32  163.40  164.21  9330200   9.330200e+06         1.000000          Hold  Normal volume; no strong signal.      0.000000                     1.92           Buy  Bullish candle (Close > Open).  Based on Open vs Close (100%)   

In [20]:
# ===== Block 4.b2 — to give top most turn signals =====
top_turn_rows = {}

for sheet_name, df_proc in processed.items():
    # Ensure Turn is sorted descending if needed
    df_proc_sorted = df_proc.sort_values(by="Turn", ascending=False)
    
    # Keep only the top row for each unique Turn
    top_rows = df_proc_sorted.groupby("Turn").head(1).reset_index(drop=True)
    
    # Store in dictionary
    top_turn_rows[sheet_name] = top_rows

# Remove rows where Turn == 6
for sheet_name in top_turn_rows:
    top_turn_rows[sheet_name] = top_turn_rows[sheet_name][top_turn_rows[sheet_name]["Turn"] != 6]

print("Top rows for each Turn:")
for sheet_name, df in top_turn_rows.items():
    print(f"\n{sheet_name}:\n", df.head())
    
with pd.ExcelWriter(r"E:\FYP\FYP Symposium\Indicator predictions\combined turns market prediction.xlsx") as writer:
    for sheet_name, df in top_turn_rows.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print("Exported each stock's top turns to separate sheets.")



Top rows for each Turn:

TSLA:
    Turn  Date    Open    High     Low   Close     Volume  Volume 7d Avg  Volume Strength Volume Signal                                 Volume Explanation  Volume Score  Volatility (High - Low) Candle Signal              Candle Explanation      Candle Signal Explanation  Candle Score  Candle Weighted Score  MACD (12,26,9)  Signal (12,26,9)  MACD Histogram (12,26,9) MACD Signal  MACD Calc (MACD - Signal)                                   MACD Explanation  MACD Score  Top Bollinger Bands (20,O,2,ma,n)  Median Bollinger Bands (20,O,2,ma,n)  Bottom Bollinger Bands (20,O,2,ma,n)  Bollinger Position % Bollinger Signal                              Bollinger Explanation  Bollinger Score  Final Score Predicted Action                             Prediction Explanation
1     5    26  357.89  372.33  356.91  364.65   71145900   6.851117e+07         1.038457          Hold                   Normal volume; no strong signal.      0.038457                    15.42        

In [21]:
# # ===== Block 4.b1 — Use Processed Data for Final Signal Prediction =====

# import pandas as pd

# # Define model functions
# def model_weighted_voting(signals, weights):
#     scores = {"Buy": 0, "Sell": 0, "Hold": 0}
#     for ind, vote in signals.items():
#         if vote == "Buy":
#             scores["Buy"] += weights.get(ind, 1.0)
#         elif vote == "Sell":
#             scores["Sell"] += weights.get(ind, 1.0)
#         else:
#             scores["Hold"] += 0.5 * weights.get(ind, 1.0)
#     if scores["Buy"] > scores["Sell"] and scores["Buy"] > scores["Hold"]:
#         return "Buy"
#     elif scores["Sell"] > scores["Buy"] and scores["Sell"] > scores["Hold"]:
#         return "Sell"
#     else:
#         return "Hold"

# def model_soft_majority(signals):
#     return max(set(signals.values()), key=list(signals.values()).count)

# def model_momentum_heavy(signals):
#     return signals["Momentum"]

# def model_volume_rsi(signals):
#     return signals["Volume"] if signals["Volume"] == signals["RSI"] else "Hold"

# def model_logical_rsi_bias(signals):
#     if signals["RSI"] == "Buy" and signals["Candle"] == "Buy":
#         return "Buy"
#     elif signals["RSI"] == "Sell" and signals["Candle"] == "Sell":
#         return "Sell"
#     return "Hold"

# # Register models for selection
# models = {
#     "Weighted Voting": model_weighted_voting,
#     "Soft Majority": model_soft_majority,
#     "Momentum Only": model_momentum_heavy,
#     "Volume + RSI": model_volume_rsi,
#     "RSI + Candle": model_logical_rsi_bias
# }

# def predict_next_move(processed, best_model_per_stock, weights):
#     final_predictions = []

#     for stock, model_name in best_model_per_stock.items():
#         model_func = models[model_name]
#         df = processed[stock][::-1].reset_index(drop=True)
#         unique_turns = sorted(df["Turn"].unique())

#         for i in range(2, len(unique_turns)):
#             if unique_turns[i] == 6:  # Skip turn 6
#                 continue

#             t_minus_2 = df[df["Turn"] == unique_turns[i - 2]]
#             t_minus_1 = df[df["Turn"] == unique_turns[i - 1]]
#             t_current = df[df["Turn"] == unique_turns[i]]

#             past_window = pd.concat([t_minus_2, t_minus_1], ignore_index=True)

#             def rsi_signal(val):
#                 if val > 70:
#                     return "Sell"
#                 elif val < 30:
#                     return "Buy"
#                 else:
#                     return "Hold"

#             past_window["RSI Signal"] = past_window["RSI (14)"].apply(rsi_signal)
#             past_window["Momentum Signal"] = past_window["Momentum (10)"].apply(
#                 lambda x: "Buy" if x > 0 else ("Sell" if x < 0 else "Hold")
#             )

#             signal_sets = {
#                 "MACD": past_window["MACD Signal"].value_counts().idxmax(),
#                 "Bollinger": past_window["Bollinger Signal"].value_counts().idxmax(),
#                 "Volume": past_window["Volume Signal"].value_counts().idxmax(),
#                 "Candle": past_window["Candle Signal"].value_counts().idxmax(),
#                 "RSI": past_window["RSI Signal"].value_counts().idxmax(),
#                 "EMA Signal": past_window["EMA Signal"].value_counts().idxmax(),
#                 "Momentum": past_window["Momentum Signal"].value_counts().idxmax()
#             }

#             if model_name == "Weighted Voting":
#                 pred = model_func(signal_sets, weights)
#             else:
#                 pred = model_func(signal_sets)

#             prior_top_avg = (t_minus_2.iloc[0]["Close"] + t_minus_1.iloc[0]["Close"]) / 2
#             actual_top_close = t_current.iloc[0]["Close"]
#             actual = "Buy" if actual_top_close > prior_top_avg else "Sell"

#             final_predictions.append({
#                 "Stock": stock,
#                 "Turn": unique_turns[i],
#                 "Predicted Action": pred,
#                 "Market": actual,
#                 "Correct": pred == actual
#             })

#     return pd.DataFrame(final_predictions)

# # Example model assignment
# best_model_per_stock = {
#     "TSLA": "Weighted Voting",
#     "XOM": "RSI + Candle",
#     "PG": "Volume + RSI",
#     "NFLX": "Weighted Voting"
# }

# # Use it
# final_df = predict_next_move(processed, best_model_per_stock, weights={
#     "MACD": 1.0, "Bollinger": 1.0, "Volume": 1.0,
#     "Candle": 1.0, "RSI": 1.0, "EMA Signal": 1.0, "Momentum": 1.0
# })
# print(final_df)


In [ ]:
# ===== Block 4.7c & 4.7d — 7-Row Turn Signal Voting for Next Turn's Top Row (Stock-Specific Weights + Market Condition) =====

import pandas as pd
from collections import defaultdict

# Define custom signal weights per stock
custom_weights = {
    "TSLA": {"MACD Signal": 0.4, "Bollinger Signal": 0.4, "Volume Signal": 0.4, "Candle Signal": 0.8},
    "XOM":  {"MACD Signal": 0.4, "Bollinger Signal": 0.4, "Volume Signal": 0.4, "Candle Signal": 0.8},
    "PG":   {"MACD Signal": 0.4, "Bollinger Signal": 0.4, "Volume Signal": 0.4, "Candle Signal": 0.6},
    "NFLX": {"MACD Signal": 0.4, "Bollinger Signal": 0.4, "Volume Signal": 0.4, "Candle Signal": 1.4}
}

# Define market condition classifier
def classify_market_state(past_avg, future_close, threshold=0.005):
    change = (future_close - past_avg) / past_avg
    if change > threshold:
        return "Attractive"
    elif change < -threshold:
        return "Risky"
    else:
        return "Cautious"

turn_results = []

for sheet_name, df_proc in processed.items():
    df_proc = df_proc.copy()
    unique_turns = sorted(df_proc["Turn"].unique())

    # Get weights for the current stock
    signal_weights = custom_weights.get(sheet_name, {
        "MACD Signal": 1.0,
        "Bollinger Signal": 1.0,
        "Volume Signal": 0.8,
        "Candle Signal": 0.6
    })

    for i in range(len(unique_turns) - 1):  # skip the last turn
        curr_turn = unique_turns[i]
        next_turn = unique_turns[i + 1]

        df_curr = df_proc[df_proc["Turn"] == curr_turn].tail(7)
        df_next = df_proc[df_proc["Turn"] == next_turn]

        if df_curr.empty or df_next.empty:
            continue

        past_close_avg = df_curr["Close"].mean()
        future_top_close = df_next.iloc[0]["Close"]  # Top of next turn

        # === Voting-Based Decision Logic ===
        scores = {"Buy": 0, "Sell": 0, "Hold": 0}

        for signal_col, weight in signal_weights.items():
            signal_counts = df_curr[signal_col].value_counts()
            for signal in ["Buy", "Sell", "Hold"]:
                scores[signal] += weight * signal_counts.get(signal, 0)

        # Final decision: signal with highest weighted score
        prediction = max(scores, key=scores.get)

        # Ground Truth Comparison
        if future_top_close > past_close_avg:
            actual = "Buy"
        elif future_top_close < past_close_avg:
            actual = "Sell"
        else:
            actual = "Hold"

        # Append row with market condition
        turn_results.append({
            "Stock": sheet_name,
            "Turn": curr_turn,
            "MACD Votes": df_curr["MACD Signal"].value_counts().to_dict(),
            "Bollinger Votes": df_curr["Bollinger Signal"].value_counts().to_dict(),
            "Volume Votes": df_curr["Volume Signal"].value_counts().to_dict(),
            "Candle Votes": df_curr["Candle Signal"].value_counts().to_dict(),
            "Prediction": prediction,
            "Past Avg Close": round(past_close_avg, 2),
            "Next Turn Top Close": round(future_top_close, 2),
            "Market": actual,
            "Market Condition": classify_market_state(past_close_avg, future_top_close),
            "Correct": prediction == actual
        })

# Organize results by stock
stock_turn_dfs = defaultdict(list)
for row in turn_results:
    stock_turn_dfs[row["Stock"]].append(row)

# Convert to DataFrames
turn_summary_dfs = {
    stock: pd.DataFrame(rows) for stock, rows in stock_turn_dfs.items()
}

# Export to Excel: separate sheet for each stock
export_path = r"E:\FYP\FYP Symposium\Indicator predictions\7row market prediction for next turn top row.xlsx"
with pd.ExcelWriter(export_path, engine="xlsxwriter") as writer:
    for stock, df in turn_summary_dfs.items():
        df.to_excel(writer, sheet_name=stock[:31], index=False)

print(f"✅ Exported to: {export_path}")

df = pd.read_excel(export_path, sheet_name=None)
for sheet_name, data in df.items():
    print(f"\nSheet: {sheet_name}")
    display(data.head(10))


✅ Exported to: E:\FYP\FYP Symposium\Indicator predictions\7row market prediction for next turn top row.xlsx

Sheet: TSLA


,Stock,Turn,MACD Votes,Bollinger Votes,Volume Votes,Candle Votes,Prediction,Past Avg Close,Next Turn Top Close,Market,Market Condition,Correct
0,TSLA,1,"{'Sell': 4, 'Buy': 3}","{'Hold': 5, 'Sell': 2}","{'Hold': 4, 'Buy': 2, 'Sell': 1}","{'Sell': 5, 'Buy': 2}",Sell,361.28,372.00,Buy,Attractive,False
1,TSLA,2,{'Sell': 7},{'Hold': 7},"{'Hold': 5, 'Buy': 2}","{'Buy': 4, 'Sell': 3}",Sell,369.48,336.34,Sell,Risky,True
2,TSLA,3,{'Sell': 7},"{'Hold': 5, 'Buy': 2}","{'Hold': 4, 'Buy': 2, 'Sell': 1}","{'Sell': 4, 'Buy': 3}",Sell,360.35,325.33,Sell,Risky,True
3,TSLA,4,{'Sell': 7},"{'Buy': 5, 'Hold': 2}","{'Hold': 6, 'Sell': 1}","{'Buy': 5, 'Sell': 2}",Buy,335.35,364.65,Buy,Attractive,True
4,TSLA,5,"{'Sell': 6, 'Buy': 1}","{'Buy': 4, 'Hold': 3}","{'Hold': 4, 'Buy': 3}","{'Buy': 5, 'Sell': 2}",Buy,327.04,362.71,Buy,Attractive,True



Sheet: XOM


,Stock,Turn,MACD Votes,Bollinger Votes,Volume Votes,Candle Votes,Prediction,Past Avg Close,Next Turn Top Close,Market,Market Condition,Correct
0,XOM,1,{'Sell': 7},"{'Hold': 4, 'Sell': 3}","{'Hold': 6, 'Sell': 1}","{'Buy': 4, 'Sell': 3}",Sell,64.83,63.48,Sell,Risky,True
1,XOM,2,{'Sell': 7},"{'Hold': 4, 'Buy': 3}","{'Hold': 4, 'Sell': 2, 'Buy': 1}","{'Buy': 4, 'Sell': 3}",Sell,63.10,61.58,Sell,Risky,True
2,XOM,3,{'Sell': 7},"{'Buy': 5, 'Hold': 2}","{'Hold': 4, 'Buy': 3}","{'Sell': 5, 'Buy': 2}",Sell,60.89,61.27,Buy,Attractive,False
3,XOM,4,"{'Buy': 4, 'Sell': 3}",{'Hold': 7},"{'Hold': 5, 'Sell': 2}","{'Sell': 4, 'Buy': 3}",Sell,62.11,61.89,Sell,Cautious,True
4,XOM,5,"{'Sell': 6, 'Buy': 1}","{'Hold': 5, 'Buy': 2}","{'Hold': 3, 'Buy': 3, 'Sell': 1}","{'Buy': 4, 'Sell': 3}",Buy,60.71,66.75,Buy,Attractive,True



Sheet: NFLX


,Stock,Turn,MACD Votes,Bollinger Votes,Volume Votes,Candle Votes,Prediction,Past Avg Close,Next Turn Top Close,Market,Market Condition,Correct
0,NFLX,1,{'Sell': 7},"{'Hold': 6, 'Sell': 1}","{'Hold': 5, 'Buy': 2}","{'Buy': 4, 'Sell': 3}",Sell,659.94,658.29,Sell,Cautious,True
1,NFLX,2,"{'Sell': 6, 'Buy': 1}","{'Hold': 3, 'Buy': 2, 'Sell': 2}","{'Hold': 5, 'Sell': 2}","{'Sell': 4, 'Buy': 3}",Sell,673.07,612.69,Sell,Risky,True
2,NFLX,3,{'Sell': 7},"{'Buy': 5, 'Hold': 2}","{'Hold': 4, 'Buy': 2, 'Sell': 1}","{'Sell': 5, 'Buy': 2}",Sell,631.49,605.04,Sell,Risky,True
3,NFLX,4,{'Sell': 7},{'Buy': 7},{'Hold': 7},"{'Sell': 5, 'Buy': 2}",Sell,611.99,613.12,Buy,Cautious,False
4,NFLX,5,"{'Sell': 5, 'Buy': 2}","{'Hold': 4, 'Buy': 3}","{'Hold': 4, 'Buy': 3}","{'Sell': 4, 'Buy': 3}",Sell,602.56,567.52,Sell,Risky,True



Sheet: PG


,Stock,Turn,MACD Votes,Bollinger Votes,Volume Votes,Candle Votes,Prediction,Past Avg Close,Next Turn Top Close,Market,Market Condition,Correct
0,PG,1,{'Buy': 7},{'Sell': 7},"{'Hold': 5, 'Sell': 1, 'Buy': 1}","{'Buy': 4, 'Sell': 3}",Buy,146.22,148.66,Buy,Attractive,True
1,PG,2,{'Buy': 7},"{'Sell': 6, 'Hold': 1}","{'Hold': 6, 'Sell': 1}","{'Buy': 4, 'Sell': 3}",Buy,147.73,152.15,Buy,Attractive,True
2,PG,3,"{'Sell': 4, 'Buy': 3}","{'Sell': 3, 'Hold': 3, 'Buy': 1}","{'Hold': 4, 'Sell': 2, 'Buy': 1}","{'Buy': 5, 'Sell': 2}",Buy,148.11,158.86,Buy,Attractive,True
3,PG,4,{'Buy': 7},{'Sell': 7},"{'Hold': 6, 'Sell': 1}","{'Buy': 6, 'Sell': 1}",Buy,155.39,161.97,Buy,Attractive,True
4,PG,5,{'Buy': 7},{'Sell': 7},"{'Hold': 4, 'Buy': 2, 'Sell': 1}","{'Buy': 6, 'Sell': 1}",Buy,159.37,164.21,Buy,Attractive,True


In [ ]:
# # ==== Block 4.7e: Experimenting with weights ====

# import pandas as pd

# for sheet_name, df_proc in processed.items():

#     # Adjust weights only — everything else is precomputed
#     weights = {"MACD": 0.34, "Bollinger": 0.31, "Volume": -0.17, "Candle": -0.69}

#     df_proc["Adjusted Final Score"] = (
#         weights["MACD"] * df_proc["MACD Score"] +
#         weights["Bollinger"] * df_proc["Bollinger Score"] +
#         weights["Volume"] * df_proc["Volume Score"] +
#         weights["Candle"] * df_proc["Candle Weighted Score"]
#     )

#     df_proc["Adjusted Action"] = df_proc["Adjusted Final Score"].apply(
#         lambda x: "Buy" if x > 1 else ("Sell" if x < -1 else "Hold")
#     )

#     # (Optional) Save back to dictionary
#     processed[sheet_name] = df_proc
    
# for sheet_name, df in processed.items():
#     print(f"\n--- {sheet_name} ---")
#     print(df.head())  # Or use df[['Date', 'Close', 'Predicted Action']].head() for specific columns


--- TSLA ---
   Turn  Date    Open    High     Low   Close     Volume  Volume 7d Avg  Volume Strength Volume Signal                Volume Explanation  Volume Score  Volatility (High - Low) Candle Signal              Candle Explanation      Candle Signal Explanation  Candle Score  Candle Weighted Score  MACD (12,26,9)  Signal (12,26,9)  MACD Histogram (12,26,9) MACD Signal  MACD Calc (MACD - Signal)                                   MACD Explanation  MACD Score  Top Bollinger Bands (20,O,2,ma,n)  Median Bollinger Bands (20,O,2,ma,n)  Bottom Bollinger Bands (20,O,2,ma,n)  Bollinger Position % Bollinger Signal                            Bollinger Explanation  Bollinger Score  Final Score Predicted Action                             Prediction Explanation  Adjusted Final Score Adjusted Action
0     6     4  382.22  390.11  360.34  362.71   80119800     80119800.0         1.000000          Hold  Normal volume; no strong signal.      0.000000                    29.77          Sell  Bearish 

In [ ]:
# ==== Block 4.7f: Actions of participants vs Market ====

import pandas as pd
from pathlib import Path

# --- Load participant decision data ---
merged_log = pd.read_excel(r"E:\FYP\FYP Symposium\Merged Log (49).xlsx")
merged_log.columns = merged_log.columns.str.strip().str.lower()
merged_log.rename(columns={"participant_id": "participant", "ticker": "stock"}, inplace=True)
action_col = next((col for col in merged_log.columns if "action" in col), None)  # auto-detect action column

# --- Load model prediction data from saved file ---
prediction_path = Path(r"E:\FYP\FYP Symposium\Indicator predictions\7row market prediction for next turn top row.xlsx")
xls = pd.ExcelFile(prediction_path)
turn_summary_dfs = {sheet: xls.parse(sheet) for sheet in xls.sheet_names}
turn_predictions = pd.concat(turn_summary_dfs.values(), ignore_index=True)
turn_predictions.columns = turn_predictions.columns.str.strip().str.lower()

merged_log = merged_log[merged_log["turn"] != 6]

def map_bias(row):
    if row["action"] == "Buy" and row["market condition"] == "Attractive":
        return -1
    elif row["action"] == "Sell" and row["market condition"] == "Attractive":
        return 1
    elif row["market condition"] == "Cautious":
        return 0
    elif row["action"] == "Sell" and row["market condition"] == "Risky":
        return -1
    elif row["action"] == "Buy" and row["market condition"] == "Risky":
        return 1
    else:
        return None  # in case of other combinations



# --- Merge datasets ---
merged_df = pd.merge(
    merged_log,
    turn_predictions[["stock", "turn", "market", "market condition"]],
    on=["stock", "turn"],
    how="left"
)

# --- Final output DataFrame ---
action_vs_market_df = merged_df[["participant", "stock", "turn", action_col, "market condition"]]
action_vs_market_df.rename(columns={action_col: "action"}, inplace=True)
action_vs_market_df["data action score"] = action_vs_market_df.apply(map_bias, axis=1)

print(action_vs_market_df.head(5))


✅ Exported to: E:\FYP\FYP Symposium\Indicator predictions\Action_vs_Market_Comparison.xlsx
  participant stock  turn action market condition  data action score
0       E0070  TSLA     1    Buy       Attractive                 -1
1       E0070  TSLA     2   Sell            Risky                 -1
2       E0070    PG     2    Buy       Attractive                 -1
3       E0070    PG     3   Sell       Attractive                  1
4       E0070  NFLX     3    Buy            Risky                  1


C:\Users\MM COMPUTERS\AppData\Local\Temp\ipykernel_34404\4028977285.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  action_vs_market_df.rename(columns={action_col: "action"}, inplace=True)
C:\Users\MM COMPUTERS\AppData\Local\Temp\ipykernel_34404\4028977285.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  action_vs_market_df["data action score"] = action_vs_market_df.apply(map_bias, axis=1)


In [ ]:
# ==== Block 4.7a: Action_vs_Market_Comparison_with_Participant_Penalty ====

# Step 1: Normalize and merge the 'Data Penalty' from strategy
export_strategy.columns = export_strategy.columns.str.strip().str.lower()
penalty_map = export_strategy[["participant_id", "data penalty"]].copy()
penalty_map.rename(columns={"participant_id": "participant"}, inplace=True)

# Drop existing 'data penalty' if already present to avoid merge errors
if "data penalty" in action_vs_market_df.columns:
    action_vs_market_df.drop(columns=["data penalty"], inplace=True)

action_vs_market_df = pd.merge(action_vs_market_df, penalty_map, on="participant", how="left")

# Step 2: Apply the conditional penalty logic
def conditional_penalty(row):
    if (row["action"] == "Buy" and row["market condition"] == "Attractive") or \
       (row["action"] == "Sell" and row["market condition"] == "Risky") or \
       (row["action"] == "Buy" and row["market condition"] == "Risky") or \
       (row["action"] == "Sell" and row["market condition"] == "Attractive"):
        return row["data penalty"]
    elif row["market condition"] == "Cautious":
        return 0
    else:
        return None

# Assign new column
action_vs_market_df["data_penalty_score"] = action_vs_market_df.apply(conditional_penalty, axis=1)


# Final DataFrame
action_vs_market_penalty_df = action_vs_market_df[[
    "participant", "stock", "turn", "action", "market condition", "data action score", "data_penalty_score"
]].copy()

action_vs_market_penalty_df["combined_score"] = (
    action_vs_market_penalty_df["data action score"] + action_vs_market_penalty_df["data_penalty_score"]
)

# Group by participant and sum combined scores
participant_summary_df = action_vs_market_penalty_df.groupby("participant", as_index=False)["combined_score"].sum()

# Normalize combined_score to a 1–5 scale
min_score = participant_summary_df["combined_score"].min()
max_score = participant_summary_df["combined_score"].max()

# Avoid division by zero
if min_score == max_score:
    participant_summary_df["normalized_score_1_5"] = 3 
else:
    participant_summary_df["normalized_score_1_5"] = (
        1 + 4 * (participant_summary_df["combined_score"] - min_score) / (max_score - min_score)
    ).round(2)

# View sample
print(action_vs_market_penalty_df.head(5))

# Export to Excel with two sheets
output_penalty_path = r"E:\FYP\FYP Symposium\Indicator predictions\Action_vs_Market_Comparison_with_Penalty.xlsx"
with pd.ExcelWriter(output_penalty_path, engine='xlsxwriter') as writer:
    action_vs_market_penalty_df.to_excel(writer, sheet_name='Action_vs_Market_Penalty', index=False)
    participant_summary_df.to_excel(writer, sheet_name='Participant_Summary', index=False)


  participant stock  turn action market condition  data action score  data_penalty_score  combined_score
0       E0070  TSLA     1    Buy       Attractive                 -1                  -1              -2
1       E0070  TSLA     2   Sell            Risky                 -1                  -1              -2
2       E0070    PG     2    Buy       Attractive                 -1                  -1              -2
3       E0070    PG     3   Sell       Attractive                  1                  -1               0
4       E0070  NFLX     3    Buy            Risky                  1                  -1               0


In [ ]:
# ========== BLOCK 4: Factual trends (MACD/Bollinger/Price/Volume signals) ==========
import pandas as pd
import os

# --- 3.1: Check if participant knows indicator
def participant_knows_indicator(prm2b_14a_response, indicator):
    if pd.isnull(prm2b_14a_response):
        return False
    return indicator.lower() in prm2b_14a_response.lower()

# --- 3.2: Detect market signals from price data
def detect_market_signals(stock_df, turn, days=5):
    visible_data = stock_df[stock_df['Turn'] <= turn].sort_values(by='Turn', ascending=False).head(days)

    price_trend = 'neutral'
    macd_signal = 'neutral'
    bb_trend = 'neutral'

    if len(visible_data) >= 3:
        close_now = visible_data.iloc[0]['Close']
        close_past = visible_data.iloc[-1]['Close']
        change_pct = (close_now - close_past) / close_past * 100

        if change_pct > 2.0:
            price_trend = 'uptrend'
        elif change_pct < -2.0:
            price_trend = 'downtrend'
        else:
            price_trend = 'neutral'

        macd = visible_data.iloc[0]['MACD (12,26,9)']
        signal = visible_data.iloc[0]['Signal (12,26,9)']
        macd_hist = visible_data.iloc[0]['MACD Histogram (12,26,9)']

        if macd > signal and macd_hist > 0:
            macd_signal = 'buy'
        elif macd < signal and macd_hist < 0:
            macd_signal = 'sell'

        close = visible_data.iloc[0]['Close']
        top_bb = visible_data.iloc[0]['Top Bollinger Bands (20,O,2,ma,n)']
        bottom_bb = visible_data.iloc[0]['Bottom Bollinger Bands (20,O,2,ma,n)']
        if close > top_bb:
            bb_trend = 'overbought'
        elif close < bottom_bb:
            bb_trend = 'oversold'
        else:
            bb_trend = 'neutral'

    return price_trend, macd_signal, bb_trend

# --- Suggest Buy Logic
def suggest_buy_decision(price_trend, macd_signal, bb_trend, volatility):
    score = 0
    if price_trend == 'uptrend': score += 1
    if macd_signal == 'buy': score += 1
    if bb_trend == 'oversold': score += 1
    if volatility < 10: score += 1
    return 'Attractive' if score >= 2 else 'Risky'

# --- Load raw stock data
experiment_file_path = r'E:\FYP\FYP Symposium\Trading Simulation Experiment Data Turn Wise.xlsx'
stock_tables = {}

for stock in ['TSLA', 'XOM', 'NFLX', 'PG']:
    df = pd.read_excel(experiment_file_path, sheet_name=stock)
    df = df.sort_values(by='Turn', ascending=False).reset_index(drop=True)
    stock_tables[stock] = df

# ========== TURN-WISE TREND METRICS CREATION ==========
file_path = r'E:\FYP\FYP Symposium\Turn Data in descending order.xlsx'
sheet_names = ['TSLA', 'NFLX', 'PG', 'XOM']
turn_data = {}

for sheet in sheet_names:
    df = pd.read_excel(file_path, sheet_name=sheet)

    # Price trend
    df['trend change'] = df['Close'] - df['Close'].shift(-1)

    def get_trend_direction(change):
        if pd.isna(change): return None
        elif change > 0: return 'uptrend'
        elif change < 0: return 'downtrend'
        else: return 'neutral'

    df['price_trend_1'] = df['trend change'].apply(get_trend_direction)

    def summarize_trend_block(group):
        up = (group['price_trend_1'] == 'uptrend').sum()
        down = (group['price_trend_1'] == 'downtrend').sum()
        group['price_trend_7'] = 'uptrend' if up > down else 'downtrend' if down > up else 'equal'
        return group

    df = df.groupby('Turn', group_keys=False).apply(summarize_trend_block)

    # Volume trend
    df['volume_diff'] = df['Volume'] - df['Volume'].shift(-1)

    def get_volume_trend_direction(change):
        if pd.isna(change): return None
        elif change > 0: return 'uptrend'
        elif change < 0: return 'downtrend'
        else: return 'neutral'

    df['volume_trend_1'] = df['volume_diff'].apply(get_volume_trend_direction)

    def summarize_volume_trend_block(group):
        up = (group['volume_trend_1'] == 'uptrend').sum()
        down = (group['volume_trend_1'] == 'downtrend').sum()
        group['volume_trend_7'] = 'uptrend' if up > down else 'downtrend' if down > up else 'equal'
        return group

    df = df.groupby('Turn', group_keys=False).apply(summarize_volume_trend_block)

    # MACD
    def make_technical_decision(row):
        try:
            macd = row['MACD (12,26,9)']
            signal = row['Signal (12,26,9)']
            hist = row['MACD Histogram (12,26,9)']
            if macd > signal > hist: return 'Buy'
            elif macd < signal and macd < hist: return 'Sell'
            else: return 'Neutral'
        except: return 'Neutral'

    df['MACD_trend'] = df.apply(make_technical_decision, axis=1)

    # Bollinger
    def classify_bollinger(row):
        try:
            if row['Close'] > row['Top Bollinger Bands (20,O,2,ma,n)']:
                return 'Over Bought'
            elif row['Close'] < row['Bottom Bollinger Bands (20,O,2,ma,n)']:
                return 'Over Sold'
            else:
                return 'Neutral'
        except:
            return 'Neutral'

    df['bollinger_trend'] = df.apply(classify_bollinger, axis=1)

    df['MACD_trend_7'] = df.groupby('Turn')['MACD_trend'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
    df['bollinger_trend_7'] = df.groupby('Turn')['bollinger_trend'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else None)

    turn_data[sheet] = df

# === Export full trend summary ===
output_path = r'E:\FYP\FYP Symposium\Outputs\Combined_Turn_With_Trend_Summary.xlsx'
with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    for stock, df in turn_data.items():
        df.to_excel(writer, sheet_name=stock, index=False)

print(f"✅ Exported successfully with trend summaries (Close & Volume) for all sheets to: {output_path}")

# === NEW: Load only Turn 1–5 top rows for enrichment ===
turn_summary_1to6 = {}
for stock in ['TSLA', 'NFLX', 'PG', 'XOM']:
    df = pd.read_excel(output_path, sheet_name=stock)
    top_rows = df.drop_duplicates(subset='Turn', keep='first')
    top_rows = top_rows[top_rows['Turn'].isin([6, 5, 4, 3, 2, 1])]
    top_rows = top_rows.sort_values(by='Turn', ascending=False)
    turn_summary_1to6[stock] = top_rows
    
# === Precompute volatility per Turn per stock from raw Turn Data ===
volatility_lookup = {}
for stock in ['TSLA', 'NFLX', 'PG', 'XOM']:
    df = pd.read_excel(r'E:\FYP\FYP Symposium\Turn Data in descending order.xlsx', sheet_name=stock)
    vol_by_turn = df.groupby('Turn')['Close'].std().reset_index()
    vol_by_turn.rename(columns={'Close': 'Turn_Volatility'}, inplace=True)
    volatility_lookup[stock] = vol_by_turn

# === Generate trend data table and file ===
def generate_trend_basis(stock_df, stock_name):
    trend_rows = []
    def map_score(val, pos_list, neg_list):
                val = str(val).lower()
                if val in pos_list:
                    return 1
                elif val in neg_list:
                    return -1
                return 0
    for turn in sorted(stock_df['Turn'].unique()):
        visible_data = stock_df[stock_df['Turn'] <= turn].sort_values(by='Turn', ascending=False).head(5)
        if visible_data.empty:
            continue
        try:
            price_trend, macd_signal, bb_trend = detect_market_signals(stock_df, turn)
            close = visible_data.iloc[0]['Close']
            high = visible_data.iloc[0]['High']
            low = visible_data.iloc[0]['Low']
            vol_df = volatility_lookup[stock_name]
            vol_row = vol_df[vol_df['Turn'] == turn]
            volatility = vol_row['Turn_Volatility'].values[0] if not vol_row.empty else None
            macd_hist = visible_data.iloc[0]['MACD Histogram (12,26,9)']
            macd_hist_strength = round(macd_hist, 4)
            volume_support = turn_summary_1to6[stock_name].set_index('Turn').loc[turn, 'volume_trend_7'] == 'uptrend'
            trend_reversal = False
            turn_list = stock_df['Turn'].sort_values(ascending=False).unique()
            turn_idx = list(turn_list).index(turn)
            if turn_idx + 1 < len(turn_list):
                next_turn = turn_list[turn_idx + 1]
                df_enriched = turn_summary_1to6.get(stock_name)
                if df_enriched is not None and turn in df_enriched['Turn'].values and next_turn in df_enriched['Turn'].values:
                    current_trend = df_enriched.set_index('Turn').loc[turn, 'price_trend_7']
                    next_trend = df_enriched.set_index('Turn').loc[next_turn, 'price_trend_7']
                    trend_reversal = current_trend != next_trend
            
            # --- Scoring each indicator (individual columns) ---
            score_price_trend = map_score(price_trend, ['uptrend'], ['downtrend'])
            score_macd_signal = map_score(macd_signal, ['buy'], ['sell'])
            score_bb_trend = map_score(bb_trend, ['oversold'], ['overbought'])

            # --- Summary trend scores ---
            summary_df = turn_summary_1to6[stock_name].set_index('Turn')
            if turn not in summary_df.index:
                print(f"⚠️ Turn {turn} not found in turn_summary for {stock_name}")
                continue
            summary = summary_df.loc[turn]

            score_volume_trend = map_score(summary['volume_trend_7'], ['uptrend'], ['downtrend'])
            score_macd_trend = map_score(summary['MACD_trend_7'], ['buy'], ['sell'])
            score_bb_summary = map_score(summary['bollinger_trend_7'], ['oversold'], ['overbought'])
            score_price_summary = map_score(summary['price_trend_7'], ['uptrend'], ['downtrend'])
            score_trend_reversal = map_score('Yes' if trend_reversal else 'No', ['Yes'], [])
            score_volume_support = map_score('Yes' if volume_support else 'No', ['Yes'], [])

            # --- Final score calculation ---
            
            score = (
                score_price_trend +
                score_macd_signal +
                score_bb_trend +
                score_volume_trend +
                score_macd_trend +
                score_bb_summary +
                score_price_summary +
                score_trend_reversal +
                score_volume_support
            )


            score = round(score, 2)  # Ensure consistent float precision
            
            # Summary trend indicators
            score += (
                map_score(turn_summary_1to6[stock_name].set_index('Turn').loc[turn, 'volume_trend_7'], ['uptrend'], ['downtrend']) +
                map_score(turn_summary_1to6[stock_name].set_index('Turn').loc[turn, 'MACD_trend_7'], ['buy'], ['sell']) +
                map_score(turn_summary_1to6[stock_name].set_index('Turn').loc[turn, 'bollinger_trend_7'], ['oversold'], ['overbought']) +
                map_score(turn_summary_1to6[stock_name].set_index('Turn').loc[turn, 'price_trend_7'], ['uptrend'], ['downtrend']) +
                map_score('Yes' if trend_reversal else 'No', ['Yes'], []) +
                map_score('Yes' if volume_support else 'No', ['Yes'], [])
            )
            # --- Final buy suggestion logic ---
            if score >= 3.0:
                buy_suggestion = 'Attractive'
            elif score >= 1.0:
                buy_suggestion = 'Cautious'
            else:
                buy_suggestion = 'Risky'

            

            trend_rows.append({
                'Turn': turn,
                'Close Price': close,
                'High Price': high,
                'Low Price': low,
                'Volatility': volatility,

                # Raw factual indicators
                'Price Trend': price_trend,
                'Score - Price Trend': score_price_trend,
                'MACD Signal': macd_signal,
                'Score - MACD Signal': score_macd_signal,
                'Bollinger Band Trend': bb_trend,
                'Score - Bollinger Band': score_bb_trend,
                'MACD Histogram': macd_hist,
                'MACD Histogram Strength': macd_hist_strength,

                # Summary trends
                'price_trend_7': summary['price_trend_7'],
                'Score - Price Trend (7)': score_price_summary,
                'volume_trend_7': summary['volume_trend_7'],
                'Score - Volume Trend (7)': score_volume_trend,
                'MACD_trend_7': summary['MACD_trend_7'],
                'Score - MACD Trend (7)': score_macd_trend,
                'bollinger_trend_7': summary['bollinger_trend_7'],
                'Score - Bollinger Trend (7)': score_bb_summary,

                # Flags
                'Trend Reversal Flag': 'Yes' if trend_reversal else 'No',
                'Score - Trend Reversal': score_trend_reversal,
                'Volume Trend Support': 'Yes' if volume_support else 'No',
                'Score - Volume Support': score_volume_support,

                # Final decision
                'Buy Confidence Score': round(score, 2),
                'Market': buy_suggestion
            })


        except Exception as e:
            print(f"⚠️ Error processing Turn {turn} in {stock_name}: {e}")
            continue

    trend_df = pd.DataFrame(trend_rows)
    
    # ✅ Normalize Buy Confidence Score to 1–5 after DataFrame is built
    if 'Buy Confidence Score' in trend_df.columns:
        min_score = trend_df['Buy Confidence Score'].min()
        max_score = trend_df['Buy Confidence Score'].max()
        if min_score != max_score:
            trend_df['Buy Confidence Score_Normalized'] = (
                (trend_df['Buy Confidence Score'] - min_score) / (max_score - min_score) * 4 + 1
            ).round(2)
        else:
            trend_df['Buy Confidence Score_Normalized'] = 3

    # === Combined Signal-Based Trend Reversal ===
    trend_df['Trend Reversal Flag'] = trend_df['price_trend_7'] != trend_df['price_trend_7'].shift(-1)
    trend_df['Trend Reversal Flag'] = trend_df['Trend Reversal Flag'].apply(lambda x: 'Yes' if x else 'No')

    trend_df['Volume Trend Support'] = trend_df['volume_trend_7'].apply(lambda x: 'Yes' if x == 'uptrend' else 'No')

    # === Strict grouped column ordering ===
    ordered_cols = [

        # Basic Info
        'Turn', 'Close Price', 'High Price', 'Low Price', 'Volatility',

        # PRICE-RELATED
        'Price Trend', 'Score - Price Trend',
        'price_trend_7', 'Score - Price Trend (7)',
        'Trend Reversal Flag', 'Score - Trend Reversal',

        # MACD-RELATED
        'MACD Signal', 'Score - MACD Signal',
        'MACD Histogram', 'MACD Histogram Strength',
        'MACD_trend_7', 'Score - MACD Trend (7)',

        # BOLLINGER-RELATED
        'Bollinger Band Trend', 'Score - Bollinger Band',
        'bollinger_trend_7', 'Score - Bollinger Trend (7)',

        # VOLUME-RELATED
        'volume_trend_7', 'Score - Volume Trend (7)',
        'Volume Trend Support', 'Score - Volume Support',

        # FINAL OUTPUT
        'Buy Confidence Score', 'Buy Confidence Score_Normalized',
        'Market'
    ]

    # Safely reorder
    trend_df = trend_df[[col for col in ordered_cols if col in trend_df.columns]]

    # === Add 7-day forward return and label ===
    trend_df = trend_df.sort_values(by='Turn')  # Ensure Turn is sorted

    if 'Close Price' in trend_df.columns:
        trend_df['future_return_7'] = trend_df['Close Price'].shift(-1) / trend_df['Close Price'] - 1
        trend_df['Label_7day'] = trend_df['future_return_7'].apply(
            lambda x: 'buy' if x > 0.02 else 'sell' if x < -0.02 else 'neutral'
        )


    return trend_df


    
# === Run and display ===
trend_tables = {}
for stock in stock_tables.keys():
    df = generate_trend_basis(stock_tables[stock], stock)
    df = df[df['Turn'] != 6]  # drop Turn 6
    trend_tables[stock] = df  # <-- this line was missing

# Display sample tables without color formatting
for stock, table in trend_tables.items():
    print(f"\nSample Trend Table for {stock}:")
    print(table.head(10))  # Show first 10 rows; adjust as needed
    
# Load Strategy sheet once at the top of BLOCK 5
strategy_path = r"E:\FYP\FYP Symposium\Outputs\Decision Factor Scoring.xlsx"
strategy_df = pd.read_excel(strategy_path, sheet_name='Strategy')

# Create lookup dictionary for Average Scoring
avg_score_dict = strategy_df.set_index('Participant_ID')['Average Scoring'].to_dict()


# ========== BLOCK 5: Enrich merged logs with trend-based market context ==========

# Load merged log (⚠️ it does NOT contain 'Average Scoring')
merged_log_path = r"E:\FYP\FYP Symposium\Renamed_Merged_Log_and_Strategy.xlsx"
merged_log_df = pd.read_excel(merged_log_path, sheet_name='Merged_Log')

# Load Strategy sheet to get Average Scoring
strategy_path = r"E:\FYP\FYP Symposium\Outputs\Decision Factor Scoring.xlsx"
strategy_df = pd.read_excel(strategy_path, sheet_name='Strategy')
avg_scores = strategy_df.set_index('Participant_ID')['Average Scoring'].to_dict()

# Enrichment function
def enrich_log_with_trends(merged_log_df, trend_tables):
    enriched_rows = []

    for idx, row in merged_log_df.iterrows():
        pid = row['Participant_ID']
        ticker = row['ticker']
        turn = row['turn']

        try:
            trend_row_df = trend_tables.get(ticker)
            if trend_row_df is None:
                print(f"⚠️ No trend data for ticker: {ticker}")
                continue

            trend_row = trend_row_df[trend_row_df['Turn'] == turn]
            if trend_row.empty:
                print(f"⚠️ Turn {turn} not found in trend table for {ticker}")
                continue

            trend_data = trend_row.iloc[0][['Market']].to_dict()
            enriched_row = row.to_dict()
            enriched_row.update(trend_data)
            enriched_rows.append(enriched_row)

        except Exception as e:
            print(f"⚠️ Error processing row {idx}: {e}")
            continue

    return pd.DataFrame(enriched_rows)

# Run enrichment
enriched_log_df = enrich_log_with_trends(merged_log_df, trend_tables)

# ========== BLOCK 7: Add Signal Alignment Score ==========
def compute_alignment_score(row):
    action = str(row['action']).strip().lower()
    signal = str(row['Market']).strip().lower()

    if signal == 'risky':
        return 1 if action == 'sell' else 0
    elif signal == 'cautious':
        return 0.5
    elif signal == 'attractive':
        return 1 if action == 'buy' else 0
    return 0

# Apply the alignment score
enriched_log_df['Signal Alignment Score'] = enriched_log_df.apply(compute_alignment_score, axis=1)

# Add explanation column for Signal Alignment Score
def explain_alignment_score(row):
    pid = row['Participant_ID']
    score = row['Signal Alignment Score']
    avg = avg_scores.get(pid, 0)

    if score == 0:
        return "Misaligned – Action contradicts market signal"
    elif score == 0.5:
        return "Partially Aligned – Somewhat matches the signal"
    elif score == 1:
        return "Fully Aligned – Matches the signal strength"
    
enriched_log_df['Signal Alignment Explanation'] = enriched_log_df.apply(explain_alignment_score, axis=1)

def infer_actual_usage(row, threshold=3.5):
    pid = row['Participant_ID']
    avg_score = avg_score_dict.get(pid, 0)
    signal_available = str(row.get('Signal Was Available', '')).lower() == 'yes'
    aligned = row.get('Signal Alignment Score', 0) >= 0.5

    claimed_use = avg_score >= threshold

    if signal_available and claimed_use and aligned:
        return "✅ Used Signal – Claimed & Aligned"
    elif signal_available and claimed_use and not aligned:
        return "❌ Ignored Signal – Claimed but not aligned"
    elif signal_available and not claimed_use and aligned:
        return "⚠️ Aligned Accidentally – Didn't claim use"
    elif not signal_available and claimed_use:
        return "⚪ No Signal – Can't assess usage"
    else:
        return "❌ No Use – No claim and no signal used"



def compute_action_signal_value(row):
    pid = row['Participant_ID']
    action = str(row['action']).strip().lower()
    signal = str(row['Market']).strip().lower()
    avg_score = avg_score_dict.get(pid, 0)  # Safe dictionary lookup

    if action == 'buy' and signal == 'risky':
        return 0
    elif action == 'sell' and signal == 'risky':
        return avg_score
    elif action == 'buy' and signal == 'cautious':
        return avg_score / 2
    elif action == 'sell' and signal == 'cautious':
        return avg_score / 2
    elif action == 'buy' and signal == 'attractive':
        return avg_score
    elif action == 'sell' and signal == 'attractive':
        return 0
    return 0  # Default fallback for unknown values

# Apply it
enriched_log_df['Action Signal Value'] = enriched_log_df.apply(compute_action_signal_value, axis=1)

# Save enriched merged log with signal score
final_export_path = r"E:\FYP\FYP Symposium\Outputs\Final_Enriched_Log_With_Signal_Score.xlsx"
enriched_log_df.to_excel(final_export_path, index=False)
print(f"✅ Final enriched log WITH signal alignment score exported to: {final_export_path}")

# ========== Export trend tables ==========

export_path = r'E:\FYP\FYP Symposium\Outputs\Factual Trend Tables.xlsx'
with pd.ExcelWriter(export_path, engine='openpyxl') as writer:
    for ticker, df in trend_tables.items():
        if not df.empty:
            df.to_excel(writer, sheet_name=ticker, index=False)

print(f"✅ Final enriched trend tables exported to: {export_path}")


C:\Users\MM COMPUTERS\AppData\Local\Temp\ipykernel_21852\206173638.py:95: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Turn', group_keys=False).apply(summarize_trend_block)
C:\Users\MM COMPUTERS\AppData\Local\Temp\ipykernel_21852\206173638.py:114: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Turn', group_keys=False).apply(summarize_volume_trend_block)
C:\Users\MM COMPUTER

✅ Exported successfully with trend summaries (Close & Volume) for all sheets to: E:\FYP\FYP Symposium\Outputs\Combined_Turn_With_Trend_Summary.xlsx

Sample Trend Table for TSLA:
   Turn  Close Price  High Price  Low Price  Volatility Price Trend  \
0   1.0       337.80      343.99     326.20   26.239602     neutral   
1   2.0       372.00      377.59     354.00   11.078977     neutral   
2   3.0       336.34      340.55     316.83   17.719387     neutral   
3   4.0       325.33      326.25     309.42   14.211373   downtrend   
4   5.0       364.65      372.33     356.91   25.304652     uptrend   

   Score - Price Trend price_trend_7  Score - Price Trend (7)  \
0                    0     downtrend                       -1   
1                    0       uptrend                        1   
2                    0     downtrend                       -1   
3                   -1       uptrend                        1   
4                    1       uptrend                        1   

  Tr

In [ ]:
#==== Block 8: Output 2 logic different logic but same inferences ====

import pandas as pd
import os
# ===== Display Settings for Clean Outputs in Jupyter =====
import pandas as pd
from IPython.display import display
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1200)
pd.set_option('display.max_colwidth', None)

# Define file paths
base_path = r"E:\FYP\FYP Symposium"
output_path = os.path.join(base_path, "Output 2", "Participant_Indicator_Segments.xlsx")

# Load data
log_path = os.path.join(base_path, "Renamed_Merged_Log_and_Strategy.xlsx")
survey_path = os.path.join(base_path, "Post-Survey (Responses).xlsx")

merged_df = pd.read_excel(log_path)
post_df = pd.read_excel(survey_path)

# Extract and rename indicator preferences
rank_cols = [col for col in post_df.columns if any(k in col.lower() for k in ['graph', 'data table', 'macd', 'bollinger'])]
preference_df = post_df[['Participant ID'] + rank_cols].copy()

rename_map = {
    col: 'Graph_Rank' for col in preference_df.columns if 'graph' in col.lower()
}
rename_map.update({
    col: 'DataTable_Rank' for col in preference_df.columns if 'data table' in col.lower()
})
rename_map.update({
    col: 'MACD_Rank' for col in preference_df.columns if 'macd' in col.lower()
})
rename_map.update({
    col: 'BB_Rank' for col in preference_df.columns if 'bollinger' in col.lower()
})
preference_df.rename(columns=rename_map, inplace=True)

# Compute trend signals
merged_df_sorted = merged_df.sort_values(by=['Participant_ID', 'ticker', 'turn'])
merged_df_sorted['Close'] = merged_df_sorted.groupby(['Participant_ID', 'ticker'])['stockportfolio_after'].shift(0)
merged_df_sorted['Prev_Close'] = merged_df_sorted.groupby(['Participant_ID', 'ticker'])['stockportfolio_after'].shift(1)
merged_df_sorted['price_diff'] = merged_df_sorted['Close'] - merged_df_sorted['Prev_Close']

def get_trend_signal(diff):
    if pd.isna(diff): return 'neutral'
    if diff > 0: return 'buy'
    elif diff < 0: return 'sell'
    return 'neutral'
merged_df_sorted['graph_trend_signal'] = merged_df_sorted['price_diff'].apply(get_trend_signal)

def get_macd_signal(diff):
    if pd.isna(diff): return 'neutral'
    elif diff > 500: return 'buy'
    elif diff < -500: return 'sell'
    return 'neutral'
merged_df_sorted['macd_signal'] = merged_df_sorted['price_diff'].apply(get_macd_signal)

merged_df_sorted['rolling_std'] = merged_df_sorted.groupby(['Participant_ID', 'ticker'])['stockportfolio_after'].transform(lambda x: x.rolling(window=3, min_periods=2).std())
def get_bb_signal(row):
    std = row['rolling_std']
    diff = row['price_diff']
    if pd.isna(std) or std < 1000: return 'neutral'
    elif diff > 0: return 'buy'
    elif diff < 0: return 'sell'
    return 'neutral'
merged_df_sorted['bb_signal'] = merged_df_sorted.apply(get_bb_signal, axis=1)

# Merge with preference
merged = pd.merge(merged_df_sorted, preference_df, left_on='Participant_ID', right_on='Participant ID', how='left')
merged['action_clean'] = merged['action'].str.lower().str.strip()

def compute_match(merged, rank_col, signal_col, action_col='action_clean', label=''):
    df = merged[merged[rank_col] == 1].copy()
    df[f'{label}_match'] = df.apply(lambda row: 1 if row[action_col] == row[signal_col] else 0, axis=1)
    score = df.groupby('Participant_ID')[f'{label}_match'].agg(['mean', 'count']).reset_index()
    score.columns = ['Participant_ID', f'{label}_Rate', f'{label}_Turns']
    return score

graph_score = compute_match(merged, 'Graph_Rank', 'graph_trend_signal', label='Graph')
macd_score = compute_match(merged, 'MACD_Rank', 'macd_signal', label='MACD')
bb_score = compute_match(merged, 'BB_Rank', 'bb_signal', label='BB')

combined = graph_score.merge(macd_score, on='Participant_ID', how='outer')
combined = combined.merge(bb_score, on='Participant_ID', how='outer')

# Fill missing values
for col in combined.columns:
    if 'Rate' in col or 'Turns' in col:
        combined[col] = combined[col].fillna("0")

# Add Segments
def segment(row):
    segments = []
    if isinstance(row['Graph_Rate'], float) and row['Graph_Rate'] >= 0.7:
        segments.append('Consistent Graph User')
    elif isinstance(row['Graph_Rate'], float):
        segments.append('Inconsistent Graph User')
    if isinstance(row['MACD_Rate'], float) and row['MACD_Rate'] >= 0.7:
        segments.append('Consistent MACD User')
    elif isinstance(row['MACD_Rate'], float):
        segments.append('Inconsistent MACD User')
    if isinstance(row['BB_Rate'], float) and row['BB_Rate'] >= 0.7:
        segments.append('Consistent BB User')
    elif isinstance(row['BB_Rate'], float):
        segments.append('Inconsistent BB User')
    return ', '.join(segments) if segments else 'No Top-Ranked Indicator Evaluated'

combined['Segment'] = combined.apply(segment, axis=1)

# Export to your folder
os.makedirs(os.path.dirname(output_path), exist_ok=True)
combined.to_excel(output_path, index=False)
print(f"Exported successfully to: {output_path}")




Exported successfully to: E:\FYP\FYP Symposium\Output 2\Participant_Indicator_Segments.xlsx


In [ ]:
#=====Block 9 Map The indicators to their preferences and score them. HL and OP and volatility should be weighted and avged out=====
import pandas as pd
from datetime import datetime


# Path to your trend table Excel file
trend_file_path = r"E:\FYP\FYP Symposium\Outputs\Factual Trend Tables.xlsx"

# Sheet names to load
sheet_names = ['TSLA', 'NFLX', 'PG', 'XOM']

# Create an empty DataFrame
indicator_map = pd.DataFrame()

# Loop through each sheet and combine data
for sheet in sheet_names:
    df = pd.read_excel(trend_file_path, sheet_name=sheet)
    df['Ticker'] = sheet  # Add ticker column
    indicator_map = pd.concat([indicator_map, df], ignore_index=True)

# --- Price + Volatility signal ---
def compute_price_vol_signal(row):
    trend = str(row.get('price_trend_7', '')).lower()
    vol = row.get('Volatility', None)

    price_score = 1 if trend == 'uptrend' else -1 if trend == 'downtrend' else 0

    if vol is None or pd.isna(vol):
        vol_score = 0
    elif vol < 10:
        vol_score = 1
    elif vol > 15:
        vol_score = -1
    else:
        vol_score = 0

    weighted_signal = 0.7 * price_score + 0.3 * vol_score
    return 'buy' if weighted_signal > 0 else 'sell'

indicator_map['PriceVol_Signal'] = indicator_map.apply(compute_price_vol_signal, axis=1)

# --- MACD + Histogram signal ---
def compute_macd_signal(row):
    macd_trend = str(row.get('MACD_trend_7', '')).lower()
    hist_strength = row.get('MACD Histogram Strength', 0)

    macd_score = 1 if macd_trend == 'buy' else -1 if macd_trend == 'sell' else 0
    hist_score = 1 if hist_strength > 0.5 else -1 if hist_strength < -0.5 else 0

    weighted_macd = 0.6 * macd_score + 0.4 * hist_score
    return 'buy' if weighted_macd > 0 else 'sell'

indicator_map['MACD_Signal'] = indicator_map.apply(compute_macd_signal, axis=1)

def score_bollinger(row):
    # Bollinger trend mapping
    bb = str(row['Bollinger Band Trend']).lower()
    bb_score = 1 if bb == 'oversold' else -1 if bb == 'overbought' else 0
    
    # Volatility normalized contribution (higher vol = less confidence)
    vol = row.get('Volatility', 0)
    vol_score = -1 if vol > 10 else (1 if vol < 5 else 0)

    # Weighted average
    final_score = (0.7 * bb_score) + (0.3 * vol_score)

    if final_score > 0.25:
        return 'Buy'
    elif final_score < -0.25:
        return 'Sell'
    else:
        return 'Neutral'

indicator_map['Bollinger_Signal'] = indicator_map.apply(score_bollinger, axis=1)

def score_volume(row):
    trend = str(row.get('volume_trend_7')).lower()

    if trend == 'uptrend':
        return 'Buy'
    elif trend == 'downtrend':
        return 'Sell'
    else:
        return 'Neutral'
        
indicator_map['Volume_Signal'] = indicator_map.apply(score_volume, axis=1)

# ===== Block 9 (continued): Add Overall Signal & Reasoning =====

# Define weights for each signal
signal_weights = {
    'PriceVol_Signal': 0.3,
    'MACD_Signal': 0.45,
    'Bollinger_Signal': 0.15,
    'Volume_Signal': 0.1
}

# Map signal text to numeric score
def signal_to_score(signal):
    signal = str(signal).strip().lower()
    if signal == 'buy':
        return 1
    elif signal == 'sell':
        return -1
    else:
        return 0

# Compute overall signal
def compute_overall_signal(row):
    score = 0
    reasons = []

    for sig_col, weight in signal_weights.items():
        sig = row.get(sig_col, '').strip().lower()
        sig_score = signal_to_score(sig)
        contrib = sig_score * weight
        score += contrib
        reasons.append(f"{sig_col.replace('_Signal', '')}: {sig} ({contrib:+.2f})")

    final = 'Buy' if score > 0 else 'Sell'
    return pd.Series([final, " | ".join(reasons)])

# Apply
indicator_map[['Overall_Signal', 'Signal_Reasoning']] = indicator_map.apply(compute_overall_signal, axis=1)

# === Optional: add per-indicator explanation columns ===
indicator_map['PriceVol_Reason'] = indicator_map.apply(lambda row:
    f"Trend: {row['price_trend_7']}, Volatility: {row['Volatility']}", axis=1)

indicator_map['MACD_Reason'] = indicator_map.apply(lambda row:
    f"Trend: {row['MACD_trend_7']}, Histogram Strength: {row['MACD Histogram Strength']}", axis=1)

indicator_map['Bollinger_Reason'] = indicator_map.apply(lambda row:
    f"Band: {row['Bollinger Band Trend']}, Volatility: {row['Volatility']}", axis=1)

indicator_map['Volume_Reason'] = indicator_map.apply(lambda row:
    f"Trend: {row['volume_trend_7']}", axis=1)

# Preview key signals
print(indicator_map[['Turn', 'Ticker', 'PriceVol_Signal',
                    'MACD_Signal', 'Bollinger_Signal']].head(15))


# Construct filename based on weights
P = signal_weights['PriceVol_Signal']
M = signal_weights['MACD_Signal']
B = signal_weights['Bollinger_Signal']
V = signal_weights['Volume_Signal']

# Format weights to avoid decimal issues in filenames
export_path = rf"E:\FYP\FYP Symposium\Output 2\Final_Weights_Inidcators_P_{P:.2f}_M_{M:.2f}_B_{B:.2f}_V_{V:.2f}.xlsx"

# Columns to keep
final_cols = ['Turn', 'Ticker', 'PriceVol_Signal', 'PriceVol_Reason', 'MACD_Signal', 'MACD_Reason', 'Bollinger_Signal', 'Bollinger_Reason', 'Volume_Signal', 'Volume_Reason', 'Overall_Signal', 'Signal_Reasoning']

# Filter only required columns per ticker and save to separate sheets
with pd.ExcelWriter(export_path, engine='openpyxl') as writer:
    for ticker, df in indicator_map.groupby('Ticker'):
        df_final = df[final_cols].copy()
        df_final.to_excel(writer, sheet_name=ticker, index=False)

print(f"✅ Clean indicator signal file exported to: {export_path}")



#======Decision factors ko dimagh men rakh kar decision lena hai=====

import pandas as pd

# File path and sheet name
file_path = r"E:\FYP\FYP Symposium\Renamed_Merged_Log_and_Strategy.xlsx"
sheet_name = 'Strategy'

# Columns to load
columns_to_use = [
    'Participant_ID',
    'Indicator_1st',
    'Indicator_2nd',
    'Indicator_3rd',
    'Indicator_4th',
    'Indicator_5th'
]

# Load only the specified columns
decision_factor_ranked = pd.read_excel(file_path, sheet_name=sheet_name, usecols=columns_to_use)

# Define the indicators we want to score
indicators = ['MACD', 'High/Low', 'Volume', 'Open/Close', 'Bollinger Bands']

# Initialize new columns with 0
for indicator in indicators:
    decision_factor_ranked[f'{indicator}_rank_score'] = 0

# Assign scores: 5 for 1st rank, down to 1 for 5th rank
score_map = {
    'Indicator_1st': 5,
    'Indicator_2nd': 4,
    'Indicator_3rd': 3,
    'Indicator_4th': 2,
    'Indicator_5th': 1
}

# Loop through rows and assign scores
for col, score in score_map.items():
    for indicator in indicators:
        decision_factor_ranked.loc[
            decision_factor_ranked[col] == indicator,
            f'{indicator}_rank_score'
        ] += score

# Calculate average score for price-based indicators: High/Low and Open/Close
decision_factor_ranked['price_rank_score'] = (
    decision_factor_ranked['High/Low_rank_score'] + decision_factor_ranked['Open/Close_rank_score']
) / 2

# Preview the result
print(decision_factor_ranked.head())

export_path = r"E:\FYP\FYP Symposium\Output 2\Decision_Factor_Ranked_Scores.xlsx"
decision_factor_ranked.to_excel(export_path, index=False)


    Turn Ticker PriceVol_Signal MACD_Signal Bollinger_Signal
0      1   TSLA            sell        sell             Sell
1      2   TSLA             buy        sell             Sell
2      3   TSLA            sell        sell             Sell
3      4   TSLA             buy        sell             Sell
4      5   TSLA             buy        sell             Sell
5      1   NFLX             buy        sell             Sell
6      2   NFLX            sell        sell             Sell
7      3   NFLX            sell        sell              Buy
8      4   NFLX             buy        sell             Sell
9      5   NFLX            sell        sell             Sell
10     1     PG             buy         buy              Buy
11     2     PG            sell         buy              Buy
12     3     PG             buy        sell             Sell
13     4     PG             buy         buy             Sell
14     5     PG             buy         buy              Buy
✅ Clean indicator signal

In [ ]:
#==== Block 10: Load merged log and strategy sheet to see indicator and preferences====#

import pandas as pd

# Load all sheets from the Indicator Map file
indicator_file = r"E:\FYP\FYP Symposium\Output 2\Indicator_Map_With_Signals_CLEAN.xlsx"
all_sheets = pd.read_excel(indicator_file, sheet_name=None)

# Combine all sheets into one DataFrame
indicator_map = pd.concat(
    [df.assign(Ticker=name) for name, df in all_sheets.items()],
    ignore_index=True
)

# Ensure consistent column naming
indicator_map.rename(columns={'Turn': 'turn'}, inplace=True)

# Load the base participant data
log_file = r"E:\FYP\FYP Symposium\Renamed_Merged_Log_and_Strategy.xlsx"
participant_cols = [
    'Date', 'Real Time', 'Simulation Time', 'Seconds left',
    'Participant_ID', 'turn', 'action', 'ticker'
]
participant_df = pd.read_excel(log_file, sheet_name='Merged_Log', usecols=participant_cols)

# Merge signal info from indicator_map using both 'turn' and 'ticker'
merged_df = participant_df.merge(
    indicator_map[['turn', 'Ticker', 'PriceVol_Signal', 'MACD_Signal', 'Bollinger_Signal', 'Volume_Signal']],
    how='left',
    left_on=['turn', 'ticker'],
    right_on=['turn', 'Ticker']
)

# Drop the duplicate 'Ticker' column
merged_df.drop(columns='Ticker', inplace=True)

# Load decision factor scores
ranked_file = r"E:\FYP\FYP Symposium\Output 2\Decision_Factor_Ranked_Scores.xlsx"
ranked_df = pd.read_excel(ranked_file)

# Drop High/Low and Open/Close rank scores
ranked_df = ranked_df.drop(columns=['High/Low_rank_score', 'Open/Close_rank_score'])

# Merge scores based on Participant_ID
final_df = merged_df.merge(ranked_df, on='Participant_ID', how='left')

# === Assign action-based signal scores ===
def compute_score(row, signal_col, score_col):
    signal = str(row[signal_col]).strip().lower()
    action = str(row['action']).strip().lower()
    rank_score = row.get(score_col, 0)

    if signal == 'neutral':
        return rank_score / 2
    elif signal == action:
        return rank_score
    else:
        return 0

# Apply the logic to each indicator
final_df['PriceVol_action_score'] = final_df.apply(lambda r: compute_score(r, 'PriceVol_Signal', 'High/Low_rank_score'), axis=1)
final_df['MACD_action_score'] = final_df.apply(lambda r: compute_score(r, 'MACD_Signal', 'MACD_rank_score'), axis=1)
final_df['Bollinger_action_score'] = final_df.apply(lambda r: compute_score(r, 'Bollinger_Signal', 'Bollinger Bands_rank_score'), axis=1)
final_df['Volume_action_score'] = final_df.apply(lambda r: compute_score(r, 'Volume_Signal', 'Volume_rank_score'), axis=1)

# Add Total Action Score by summing all individual action scores
final_df['Total_action_score'] = (
    final_df['PriceVol_action_score'] +
    final_df['MACD_action_score'] +
    final_df['Bollinger_action_score'] +
    final_df['Volume_action_score']
)

final_df = final_df[final_df['turn'] != 6]

# Add explanation for Total_action_score
def explain_action_alignment(row):
    scores = {
        'PriceVol': row['PriceVol_action_score'],
        'MACD': row['MACD_action_score'],
        'Bollinger': row['Bollinger_action_score'],
        'Volume': row['Volume_action_score']
    }
    strong_matches = [
        k for k, v in scores.items()
        if v >= 0.75 and str(row[f"{k}_Signal"]).strip().lower() != 'neutral'
    ]

    neutral_matches = [
        k for k, v in scores.items()
        if v >= 0.5 and str(row[f"{k}_Signal"]).strip().lower() == 'neutral'
    ]
    misses = [k for k, v in scores.items() if v == 0]

    explanation = []
    explanation = []
    if strong_matches:
        explanation.append(f"Strong alignment with {', '.join(strong_matches)}")
    if neutral_matches:
        explanation.append(f"Partial alignment with {', '.join(neutral_matches)}")
    if misses:
        explanation.append(f"Ignored or mismatched {', '.join(misses)}")

    return " | ".join(explanation)

final_df['Action_Score_Explanation'] = final_df.apply(explain_action_alignment, axis=1)

#Pulled the three columns from BLOCK 3 about knowledge from pre data if they know how to use what indicator

def check_knowledge(prm2b_14a_response):
    if pd.isna(prm2b_14a_response):
        return {'MACD': False, 'Bollinger': False, 'Volume': False, 'HighLow_OpenClose': False}

    text = prm2b_14a_response.lower()
    
    knowledge = {
        'MACD': False,
        'Bollinger': False,
        'Volume': False,
        'HighLow_OpenClose': False
    }

    # Check MACD knowledge
    if 'macd' in text:
        knowledge['MACD'] = True
    
    # Check Bollinger Bands
    if 'bollinger' in text or 'bb' in text:
        knowledge['Bollinger'] = True

    # Check Volume
    if 'volume' in text:
        knowledge['Volume'] = True

    # Check High Low Open Close
    if 'high' in text or 'low' in text or 'open' in text or 'close' in text or 'ohlc' in text:
        knowledge['HighLow_OpenClose'] = True

    return knowledge

# --- Apply to all participants ---

knowledge_records = []

for idx, row in strategy.iterrows():
    participant_id = row['Participant_ID']
    prm2b_14a = row.get('PRM2b_14a', None)
    knowledge = check_knowledge(prm2b_14a)

    knowledge_records.append({
        'Participant_ID': participant_id,
        'Knows_MACD': 'Yes' if knowledge['MACD'] else 'No',
        'Knows_Bollinger': 'Yes' if knowledge['Bollinger'] else 'No',
        'Knows_Volume': 'Yes' if knowledge['Volume'] else 'No',
        'Knows_HighLow_OpenClose': 'Yes' if knowledge['HighLow_OpenClose'] else 'No'
    })

# Create DataFrame
knowledge_df = pd.DataFrame(knowledge_records)

# Drop 'Knows_HighLow_OpenClose' — no longer needed
if 'Knows_HighLow_OpenClose' in knowledge_df.columns:
    knowledge_df.drop(columns=['Knows_HighLow_OpenClose'], inplace=True)

# --- Merge Cleaned Knowledge into Strategy ---

# Drop old versions to avoid MergeError
strategy.drop(columns=['Knows_MACD', 'Knows_Bollinger', 'Knows_Volume', 'Knows_HighLow_OpenClose', 'Knowledge_Score'], errors='ignore', inplace=True)

# Merge new knowledge
strategy = pd.merge(strategy, knowledge_df, on='Participant_ID', how='left')
print("\n✅ Merged updated knowledge into strategy successfully.")

# --- Scoring Logic ---

# Map Yes/No with new scoring logic (Volume: Yes = 1, No = 0.5)
knowledge_df_numeric = knowledge_df.copy()
knowledge_df_numeric['Knows_MACD'] = knowledge_df_numeric['Knows_MACD'].map({'Yes': 1, 'No': 0})
knowledge_df_numeric['Knows_Bollinger'] = knowledge_df_numeric['Knows_Bollinger'].map({'Yes': 1, 'No': 0})
knowledge_df_numeric['Knows_Volume'] = knowledge_df_numeric['Knows_Volume'].map({'Yes': 1, 'No': 0.5})

# Compute score
knowledge_df['Knowledge_Score'] = (
    knowledge_df_numeric['Knows_MACD'] +
    knowledge_df_numeric['Knows_Bollinger'] +
    knowledge_df_numeric['Knows_Volume']
)

# Show sample output
print("\n✅ Final Knowledge Score (range 0.5–3.0):")
display(knowledge_df[['Participant_ID', 'Knows_MACD', 'Knows_Bollinger', 'Knows_Volume', 'Knowledge_Score']].head())

# Optional documentation string
knowledge_score_note = "Knowledge_Score = Knows_MACD (1/0) + Knows_Bollinger (1/0) + Knows_Volume (1 if Yes, 0.5 if No); Range = 0.5 to 3.0"


final_df = final_df.merge(strategy[['Participant_ID', 'Knows_MACD', 'Knows_Bollinger', 'Knows_Volume']], on='Participant_ID', how='left')

def compute_used_know_score(row, signal_col, know_col):
    signal = str(row[signal_col]).strip().lower()
    action = str(row['action']).strip().lower()
    knows = row[know_col]

    if signal == action and knows == 'Yes':
        return 1
    else:
        return 0

# Apply logic for each indicator
final_df['used_MACD_Know_Score'] = final_df.apply(lambda r: compute_used_know_score(r, 'MACD_Signal', 'Knows_MACD'), axis=1)
final_df['used_Bollinger_Know_Score'] = final_df.apply(lambda r: compute_used_know_score(r, 'Bollinger_Signal', 'Knows_Bollinger'), axis=1)
final_df['used_Volume_Know_Score'] = final_df.apply(lambda r: compute_used_know_score(r, 'Volume_Signal', 'Knows_Volume'), axis=1)

# Optional: total informed usage score
final_df['Total_Used_Know_Score'] = (
    final_df['used_MACD_Know_Score'] +
    final_df['used_Bollinger_Know_Score'] +
    final_df['used_Volume_Know_Score']
)

final_df['Combined_Action_Knowledge_Score'] = (
    final_df['Total_action_score'] + final_df['Total_Used_Know_Score']
)


print(final_df[['Participant_ID', 'turn', 'ticker', 'action',
                'PriceVol_action_score', 'MACD_action_score',
                'Bollinger_action_score', 'Volume_action_score',
                'Total_action_score', 'Knows_MACD', 'Knows_Bollinger', 'Knows_Volume',
                'used_MACD_Know_Score', 'used_Bollinger_Know_Score', 'used_Volume_Know_Score', 'Combined_Action_Knowledge_Score']].head(2))

# Export the final dataframe
final_df.to_excel(
    r"E:\FYP\FYP Symposium\Output 2\participant_indicator_preference_with_signals.xlsx",
    index=False
)

print(f"✅ Final participant-indicator preference file with action scores exported to: {export_path}")



✅ Merged updated knowledge into strategy successfully.

✅ Final Knowledge Score (range 0.5–3.0):


,Participant_ID,Knows_MACD,Knows_Bollinger,Knows_Volume,Knowledge_Score
0,E0070,No,No,No,0.5
1,E0169,Yes,Yes,Yes,3.0
2,E0426,No,No,No,0.5
3,E0712,No,No,No,0.5
4,E1130,Yes,No,No,1.5


  Participant_ID  turn ticker action  PriceVol_action_score  \
0          E0070     1   TSLA    Buy                      0   
1          E0070     2   TSLA   Sell                      0   

   MACD_action_score  Bollinger_action_score  Volume_action_score  \
0                  0                       0                    4   
1                  1                       2                    0   

   Total_action_score Knows_MACD Knows_Bollinger Knows_Volume  \
0                   4         No              No           No   
1                   3         No              No           No   

   used_MACD_Know_Score  used_Bollinger_Know_Score  used_Volume_Know_Score  \
0                     0                          0                       0   
1                     0                          0                       0   

   Combined_Action_Knowledge_Score  
0                                4  
1                                3  
✅ Final participant-indicator preference file with action sc

In [ ]:
########################## # BLOCK 11: Assign Rationality (Final Version)
# --- Merge Extracted Knowledge into Strategy ---

# Assuming 'knowledge_df' already created
# Merge knowledge_df with strategy on 'Participant_ID'
# Drop old columns if exist to avoid suffix errors
cols_to_drop = ['Knows_MACD', 'Knows_Bollinger', 'Knows_Volume', 'Knows_HighLow_OpenClose']
strategy = strategy.drop(columns=[col for col in cols_to_drop if col in strategy.columns], errors='ignore')

# Now merge

strategy = pd.merge(strategy, knowledge_df, on='Participant_ID', how='left')

print("\n✅ Merged Participant Knowledge into Strategy Frame.")

def assign_rationality(row):
    participant_id = row['Participant_ID']
    action = str(row['action']).lower()
    ticker = str(row['ticker']).upper()
    turn = row['turn']

    participant_row = strategy[strategy['Participant_ID'] == participant_id]
    if participant_row.empty:
        return pd.Series(['Unknown', np.nan, np.nan, np.nan, np.nan])

    decision_factors = [
        (participant_row['DecisionFactor_1st'].values[0], 0.5),
        (participant_row['DecisionFactor_2nd'].values[0], 0.3),
        (participant_row['DecisionFactor_3rd'].values[0], 0.2)
    ]
    indicators = [
        (participant_row['Indicator_1st'].values[0], 0.5),
        (participant_row['Indicator_2nd'].values[0], 0.4),
        (participant_row['Indicator_3rd'].values[0], 0.3),
        (participant_row['Indicator_4th'].values[0], 0.2),
        (participant_row['Indicator_5th'].values[0], 0.1)
    ]
    known_indicators = participant_row['PRM2b_14a'].values[0]

    trend_table = trend_tables.get(ticker)
    if trend_table is None:
        return pd.Series(['Unknown', np.nan, np.nan, np.nan, np.nan])

    trend_row = trend_table[trend_table['Turn'] == turn]
    if trend_row.empty:
        return pd.Series(['Unknown', np.nan, np.nan, np.nan, np.nan])

    trend_row = trend_row.iloc[0]

    # Step 1: Emotionally Driven
    if str(row['news_truth']).lower() == 'false':
        if (str(row['news_sentiment']).lower() == 'positive' and action == 'buy') or \
           (str(row['news_sentiment']).lower() == 'negative' and action == 'sell'):
            return pd.Series(['Emotionally Driven', 0.0, 0.0, 0.0, 0.0])

    # Step 2: Rationality Scoring
    rationality_score = 0
    breakdown_scores = {
        'DecisionFactor_Score': 0,
        'Indicator_Score': 0,
        'Knowledge_Bonus': 0
    }

    for factor, weight in decision_factors:
        if str(factor).lower() == 'graph':
            if (trend_row['Price Trend'] == 'uptrend' and action == 'buy') or \
               (trend_row['Price Trend'] == 'downtrend' and action == 'sell'):
                rationality_score += weight
                breakdown_scores['DecisionFactor_Score'] += weight
        elif str(factor).lower() == 'news':
            if (row['news_sentiment'].lower() == 'positive' and row['news_truth'].lower() == 'true' and action == 'buy') or \
               (row['news_sentiment'].lower() == 'negative' and row['news_truth'].lower() == 'true' and action == 'sell'):
                rationality_score += weight
                breakdown_scores['DecisionFactor_Score'] += weight

    for indicator, weight in indicators:
        if pd.isna(indicator):
            continue
        if str(indicator).lower() == 'volume':
            if trend_row['volume_trend_7'].lower() == 'uptrend' and action == 'buy':
                rationality_score += weight
                breakdown_scores['Indicator_Score'] += weight
                if participant_row['Knows_Volume'].values[0] == 'Yes':
                    rationality_score += 0.2
                    breakdown_scores['Knowledge_Bonus'] += 0.2
        elif str(indicator).lower() == 'macd':
            if (trend_row['MACD Signal'] == 'buy' and action == 'buy') or (trend_row['MACD Signal'] == 'sell' and action == 'sell'):
                rationality_score += weight
                breakdown_scores['Indicator_Score'] += weight
                if participant_row['Knows_MACD'].values[0] == 'Yes':
                    rationality_score += 0.2
                    breakdown_scores['Knowledge_Bonus'] += 0.2
        elif str(indicator).lower() == 'bollinger bands':
            if (trend_row['Bollinger Band Trend'] == 'oversold' and action == 'buy') or \
               (trend_row['Bollinger Band Trend'] == 'overbought' and action == 'sell'):
                rationality_score += weight
                breakdown_scores['Indicator_Score'] += weight
                if participant_row['Knows_Bollinger'].values[0] == 'Yes':
                    rationality_score += 0.2
                    breakdown_scores['Knowledge_Bonus'] += 0.2

    # Step 3: Label
    if rationality_score >= 0.7:
        label = 'Fully Rational'
    elif 0.4 <= rationality_score < 0.7:
        label = 'Slightly Rational'
    else:
        label = 'Irrational'

    return pd.Series([
        label,
        rationality_score,
        breakdown_scores['DecisionFactor_Score'],
        breakdown_scores['Indicator_Score'],
        breakdown_scores['Knowledge_Bonus']
    ])
merged_log[['rationality_label', 'rationality_score', 'DecisionFactor_Score', 'Indicator_Score', 'Knowledge_Bonus']] = merged_log.apply(assign_rationality, axis=1)


print("\n✅ Rationality Label and Score Assigned to merged_log.")

from openpyxl import load_workbook
from datetime import datetime

# Generate Timestamp Name
current_time = datetime.now().strftime("%Y-%m-%d %H-%M-%S")
output_path = rf'E:\FYP\FYP Symposium\Outputs\Behavioral_Rationality_Log_{current_time}.xlsx'

# Save merged_log
merged_log.to_excel(output_path, index=False)

print(f"\n✅ Behavioral Rationality Log Saved Successfully at {output_path}")

# Now prepare Market Signals + Rationality
# Now prepare Market Signals + Rationality
# Prepare Market Signals + Rationality
market_signal_records = []

for idx, row in merged_log.iterrows():
    ticker = str(row['ticker']).upper()
    turn = row['turn']
    participant_id = row['Participant_ID']
    rationality = row['rationality_label']
    rationality_score = row['rationality_score']

    trend_table = trend_tables.get(ticker)
    if trend_table is None:
        continue

    trend_row = trend_table[trend_table['Turn'] == turn]
    if trend_row.empty:
        continue

    trend_row = trend_row.iloc[0]
    market_signal_records.append({
        'Participant_ID': participant_id,
        'Turn': turn,
        'Ticker': ticker,
        'Volatility': trend_row['Volatility'],
        'Price Trend': trend_row['Price Trend'],
        'MACD Signal': trend_row['MACD Signal'],
        'MACD Histogram': trend_row['MACD Histogram'],
        'Volume Trend': trend_row['volume_trend_7'],
        'Bollinger Band Trend': trend_row['Bollinger Band Trend'],
        'Market': trend_row['Market'],
        'DecisionFactor_Score': row['DecisionFactor_Score'],
        'Indicator_Score': row['Indicator_Score'],
        'Knowledge_Bonus': row['Knowledge_Bonus'],
        'Rationality Score': rationality_score,
        'Rationality Label': rationality
    })


market_signals_df = pd.DataFrame(market_signal_records)
# Now open in append mode to add new sheet
with pd.ExcelWriter(output_path, engine='openpyxl', mode='a') as writer:
    market_signals_df.to_excel(writer, sheet_name='MarketSignals_WithRationality', index=False)

print("\n✅ Market Signals + Rationality Sheet Saved Successfully!")

# --- Create Participant Behavioral Summary ---

# --- Create Full Participant Behavioral Summary ---

summary_records = []

participants = merged_log['Participant_ID'].unique()

for pid in participants:
    participant_data = merged_log[merged_log['Participant_ID'] == pid]
    total_decisions = len(participant_data)

    fully_rational = len(participant_data[participant_data['rationality_label'] == 'Fully Rational'])
    slightly_rational = len(participant_data[participant_data['rationality_label'] == 'Slightly Rational'])
    irrational = len(participant_data[participant_data['rationality_label'] == 'Irrational'])
    emotionally_driven = len(participant_data[participant_data['rationality_label'] == 'Emotionally Driven'])

    summary_records.append({
        'Participant_ID': pid,
        'Total Actions': total_decisions,
        '# Fully Rational': fully_rational,
        '# Slightly Rational': slightly_rational,
        '# Irrational': irrational,
        '# Emotionally Driven': emotionally_driven,
        '% Fully Rational': round((fully_rational / total_decisions) * 100, 2),
        '% Slightly Rational': round((slightly_rational / total_decisions) * 100, 2),
        '% Irrational': round((irrational / total_decisions) * 100, 2),
        '% Emotionally Driven': round((emotionally_driven / total_decisions) * 100, 2),
    })

# Create DataFrame
summary_df = pd.DataFrame(summary_records)

# Determine Overall Participant Classification
def classify_participant(row):
    labels = {
        'Fully Rational': row['% Fully Rational'],
        'Slightly Rational': row['% Slightly Rational'],
        'Irrational': row['% Irrational'],
        'Emotionally Driven': row['% Emotionally Driven']
    }
    dominant = max(labels, key=labels.get)
    return dominant

summary_df['Overall Classification'] = summary_df.apply(classify_participant, axis=1)

# Save to the Excel file in a new sheet
with pd.ExcelWriter(output_path, engine='openpyxl', mode='a') as writer:
    summary_df.to_excel(writer, sheet_name='Participant_Summary', index=False)

print("\n✅ Participant Behavioral Summary Sheet Saved Successfully!")
print("\n📊 FINAL OUTPUT: Top 10 Rows of Market Signals + Rationality Data:\n")
print(market_signals_df.head(10).to_string(index=False))


✅ Merged Participant Knowledge into Strategy Frame.

✅ Rationality Label and Score Assigned to merged_log.

✅ Behavioral Rationality Log Saved Successfully at E:\FYP\FYP Symposium\Outputs\Behavioral_Rationality_Log_2025-05-03 07-56-21.xlsx

✅ Market Signals + Rationality Sheet Saved Successfully!

✅ Participant Behavioral Summary Sheet Saved Successfully!

📊 FINAL OUTPUT: Top 10 Rows of Market Signals + Rationality Data:

Participant_ID  Turn Ticker  Volatility Price Trend MACD Signal  MACD Histogram Volume Trend Bollinger Band Trend     Market  DecisionFactor_Score  Indicator_Score  Knowledge_Bonus  Rationality Score  Rationality Label
         E0070     1   TSLA   26.239602     neutral     neutral           -7.68      uptrend              neutral      Risky                   0.0              0.4              0.0                0.4  Slightly Rational
         E0070     2   TSLA   11.078977     neutral     neutral           -2.91      uptrend              neutral Attractive           

In [ ]:
##########################
# BLOCK 12: Compute Final Decision Quality Score (Rationality Proxy)
##########################

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Load files
pre_survey = pd.read_excel(r"E:\FYP\FYP Symposium\Pre-Survey Form (Responses).xlsx")
post_survey = pd.read_excel(r"E:\FYP\FYP Symposium\Post-Survey (Responses).xlsx")
merged_log = pd.read_excel(r"E:\FYP\FYP Symposium\Renamed_Merged_Log_and_Strategy.xlsx")

# Clean column headers
pre_survey.columns = pre_survey.columns.str.strip()
post_survey.columns = post_survey.columns.str.strip()
merged_log.columns = merged_log.columns.str.strip()

# Rename pre-survey columns
pre_survey = pre_survey.rename(columns={
    'Participant ID (AXXXX format)\nIf ID is not assigned yet, kindly contact researchers before proceeding': 'Participant_ID',
    'How confident are you in your ability to make profitable trades?': 'Confidence',
    'When faced with Financial decisions, do you usually rely on Logic or Intuition?': 'Logic_vs_Intuition',
    'How comfortable are you with taking risks?': 'Risk_Tolerance',
    'How comfortable are you with uncertainty in Financial Markets?': 'Comfort_with_Uncertainty',
    'Do you consider yourself patient or impulsive when making decisions?': 'Impulsiveness'
})

# Select pre-survey features
pre_features = pre_survey[['Participant_ID', 'Confidence', 'Logic_vs_Intuition', 'Risk_Tolerance',
                           'Comfort_with_Uncertainty', 'Impulsiveness']]

# Rename and select post-survey columns
post_survey = post_survey.rename(columns={
    'Participant ID': 'Participant_ID',
    'How much pressure did you feel during the experiment to perform well?': 'Pressure',
    'How well do you think you have performed in the trading simulation?': 'Self_Performance',
    'How well do you think you may have performed as compared to other participants?': 'Relative_Performance'
})
post_features = post_survey[['Participant_ID', 'Pressure', 'Self_Performance', 'Relative_Performance']]

# Behavioral summary
log_summary = merged_log.groupby('Participant_ID').agg(
    Total_Trades=('action', 'count'),
    Unique_Tickers=('ticker', pd.Series.nunique)
).reset_index()

# Final cash (profit proxy) — last row per participant
profit_summary = merged_log.sort_values(by=['Participant_ID', 'turn']).groupby('Participant_ID').tail(1)[
    ['Participant_ID', 'cash_after']
].rename(columns={'cash_after': 'Final_Cash'})

# Merge all features
merged_df = pre_features.merge(post_features, on='Participant_ID', how='inner')
merged_df = merged_df.merge(log_summary, on='Participant_ID', how='inner')
merged_df = merged_df.merge(profit_summary, on='Participant_ID', how='left')

# Log-transform cash
merged_df['Log_Cash'] = np.log(merged_df['Final_Cash'])

# Normalize using MinMaxScaler
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(merged_df.drop(columns=['Participant_ID', 'Final_Cash']))
scaled_df = pd.DataFrame(scaled_data, columns=merged_df.columns.drop(['Participant_ID', 'Final_Cash']))
scaled_df['Participant_ID'] = merged_df['Participant_ID']
scaled_df['Final_Cash'] = merged_df['Final_Cash']  # Keep original value

# Compute Final Decision Quality Score
scaled_df['Final_Decision_Quality_Score'] = (
    0.25 * scaled_df[['Confidence', 'Logic_vs_Intuition', 'Risk_Tolerance',
                      'Comfort_with_Uncertainty', 'Impulsiveness']].mean(axis=1) +
    0.20 * scaled_df[['Pressure', 'Self_Performance', 'Relative_Performance']].mean(axis=1) +
    0.15 * scaled_df[['Total_Trades', 'Unique_Tickers']].mean(axis=1) +
    0.40 * scaled_df['Log_Cash']
)

# ================================
# Example Categorization: Rationality Label
# ================================
scaled_df['Rationality_Label'] = pd.cut(
    scaled_df['Final_Decision_Quality_Score'],
    bins=[0, 0.4, 0.7, 1.0],
    labels=['Irrational', 'Moderate', 'Rational']
)
# Add Explanation based on Rationality Label and Score
def explain_rationality(score, label):
    if score >= 0.85:
        return "Highly rational; confident, logical, and performed strongly under pressure with high profit."
    elif score >= 0.70:
        return "Generally rational; showed strong strategy use and psychological stability."
    elif score >= 0.55:
        return "Moderately effective; balanced decision-making but either cautious or inconsistent performance."
    elif score >= 0.40:
        return "Somewhat impulsive or stressed; moderate trading performance and mixed self-perception."
    else:
        return "Likely struggled under pressure; low confidence, impulsive patterns, or poor final performance."

scaled_df['Rationality_Explanation'] = scaled_df.apply(
    lambda row: explain_rationality(row['Final_Decision_Quality_Score'], row['Rationality_Label']),
    axis=1
)

# Add breakdown explanation per participant
def build_score_formula(row):
    cog = round(row[['Confidence', 'Logic_vs_Intuition', 'Risk_Tolerance',
                     'Comfort_with_Uncertainty', 'Impulsiveness']].mean(), 2)
    perc = round(row[['Pressure', 'Self_Performance', 'Relative_Performance']].mean(), 2)
    behav = round(row[['Total_Trades', 'Unique_Tickers']].mean(), 2)
    perf = round(row['Log_Cash'], 2)

    return f"Score = 25% Cognitive ({cog}) + 20% Perception ({perc}) + 15% Behavior ({behav}) + 40% Log-Cash ({perf})"

scaled_df['Score_Breakdown'] = scaled_df.apply(build_score_formula, axis=1)


# Save results
scaled_df[['Participant_ID', 'Final_Decision_Quality_Score', 'Rationality_Label', 'Rationality_Explanation', 'Score_Breakdown']].to_excel(
    r"E:\FYP\FYP Symposium\Output 2\Final_Decision_Quality_Scores.xlsx", index=False)

print("✅ Final Decision Quality Scores (with final cash) computed and saved.")

✅ Final Decision Quality Scores (with final cash) computed and saved.
